### RAS-Commander 1.0 (GUI Version)
- Parallel Execute Local + Remote Machines using Psexec
- Build Plans from DSS (HMS>RAS1D, RAS1D>RAS2D)
- Override Infiltration Base Parameters by CSV

Author: William (Bill) Katzenmeyer, P.E., C.F.M. (C.H. Fenstermaker and Associates, LLC) 

Notable Contributions: Sean Micek, P.E. Prototype Infiltration HDF Scaling (C.H. Fenstermaker and Associates, LLC)

Source: https://github.com/billk-FM/HEC-Commander-Tools

In [81]:
#1  -------- USER INPUTS (These inputs are defaults and are persistent) --------
#             GUI can override these settings but will not save them

# Operation Mode 
#Operation_Mode = "Run Missing" # "Run Missing" will run an existing project (only running plans that are missing from the HEC-RAS output folder)
Operation_Mode = "Build from DSS" # "Build from DSS" will build plans from HEC-RAS template folder and run them
# For now, Run Missing mode only supports files matching the naming convention created by "Build from DSS" mode

HECRAS_project_folder = r"C:\Your_HECRAS_Project Folder" # Results go into this folder 
# in Run Missing mode, this is the HEC-RAS project you wish to execute (results go back into this folder)
# in Build from DSS mode, this folder will be overwritten and new plans will be created

# Define Remote Target Folders
# These are temporary folders used to execute parallel runs
HECRAS_Deploy_Targets = [ 
r"C:\Local_Path\Temp_May_2D",
r"\\NetworkName1\Temp\Temp_May_2D",
r"\\NetworkName2\Temp\Temp_May_2D",
r"\\NetworkName3\Temp\Temp_May_2D",
r"\\NetworkName4\Temp\Temp_May_2D",
r"\\NetworkName5\Temp\Temp_May_2D",
r"\\NetworkName6\Temp\Temp_May_2D",
r"\\NetworkName7\Temp\Temp_May_2D",
#r"\\NetworkName8\Temp\Temp_May_2D",
#r"\\NetworkName9\Temp\Temp_May_2D",
#r"\\NetworkName10\Temp\Temp_May_2D",

# Add additional remote targets as needed
]  

# Accepts any combination of local and remote paths for deployment

# Define number of copies in each target (usually 2-4)
Number_Parallel_Runs = 4


# --------  IF Operation_Mode = "Build from DSS" then define the Template and DSS files used to build new plans  --------
# HEC-RAS Template Folder used to build new plans 
HECRAS_template_folder = r"C:\Your_HECRAS_Template_Folder"
# Plan Number to use for building new HEC-RAS plans (this plan will not be executed)
Plan_Number = "02"

# Folder containing DSS files to import into HEC-RAS project
DSS_Source_Folder = r"C:\Your_HECHMS_Output_DSS_Folder_or_RAS_1D_Output_Folder"
DSS_Search_Word = "RAS1D"   # Search Word for DSS files to import into HEC-RAS project
DSS_Replace_Word = "RAS2D"  # Replaces search word in HEC-RAS DSS outputs to prevent overwriting DSS inputs
# Search and Replace Word prevents RAS DSS output from overwriting DSS input files.

# OPTIONAL Prefix for filtering file names for import (default is blank, returns all files)
# Useful if you have DSS files for multiple events in the same folder
DSS_File_Name_Filter_Word = "Event"


# --------  Infiltration Layer Inputs (Currently only works with Build With DSS Mode) --------
Enable_Infiltration_Overrides = True

Infiltration_From_RASMapper_csv = r"c:\Path_To_Your\Example_Infiltration_From_RASMapper.csv"
# Copy/paste infiltration data from RAS Mapper into a CSV file.  This will be used to build the scaled infiltration layer in HEC-RAS.  See Example CSV file for formatting.

# To scale 2D impervious grids, you must provide the unscaled infiltration data table in CSV format (Copy/paste if this is already in HEC-RAS)
user_calibration_runs_csv_fullpath = r"c:\Path_To_Your\Example_User_Run_Parameters.csv"
# NOTE:  An infiltration calibration region must be present in the geometry to implmement Base Overrides.  Shape/size of polygon does not matter, but it must be present.


In [82]:
#2 -------- Additional Settings, Paths and Variables


# NOTE: Keep the first 2 letters of the folder name in HECRAS_project_folder the same as the HECRAS_template_folder.
# There is a safety mechanism that prevents the script from modifying/deleting files that don't match the safety prefix

# --------  HEC-RAS Version, Local Paths and File Type Exclusions   --------
# Specify HEC-RAS EXE Path for .bat file creation (Change Version here if needed)
hecras_exe_path = r"C:\Program Files (x86)\HEC\HEC-RAS\6.2\Ras.exe"
print ("HEC-RAS Executable Path Used for batch file creation: ", hecras_exe_path) 


# --------  ** PSEXEC Control Options **
# Additional capabilities are avilable if running system account, such as background execution 
# Flag to run PSEXEC in system account (Yes or No) (uses -s flag instead of -i {Psexec_Session_ID})
Psexec_Run_In_System_Account = "No"  # "Yes" or "No"

# If not in system account, specify session ID to run in (usually 2 for single user machines) (-i {Psexec_Session_ID})
# Task Manager detail tab can be used to find Session ID on remote machine
Psexec_Session_ID = 2 

# Remote Deployment Task Priority (low, below normal, normal)
Psexec_Priority = "low" # low recommended for maximum responsiveness on multi-user machines

# Local Target Path used in Batch File Creation 
Remote_Share_Path = "C:\\"

# Build Remote_Base_Directory for batch file creation
Remote_Base_Directory = Remote_Share_Path + HECRAS_Deploy_Targets[0].split('\\', 3)[-1]
print ("Local Target Path Used for batch file creation: ", Remote_Base_Directory) 

# Folder Folder_Safety_Prefix:  This prevents the script from deleting unrelated folders 
# Specify how many of the folder prefix characters need to match  
Folder_Safety_Prefix_Length = 2

# Define exclusions for folder deployment 
# NOTE: .tif, .tiff are needed for 2D preprocessing Example for 1D models "'*.tif', '*.tiff', '*.img', '*.p**.hdf'"
exclusions = []
print ("The following file types are not needed for 1D compute runs and will be excluded: ", exclusions)

# Number of Parallel Threads for File Copy Operations
File_Copy_Threads = 4
print ("Number of Parallel Threads for File Copy Operations: ", File_Copy_Threads)

# If using a simple file share ("\\server\share"), give the local path to the share here
# Needs to be the same for all machines
Remote_File_Share_Path = "C:"
print ("Remote File Share Path: ", Remote_File_Share_Path)

# if using HMS as the input, the run name will not change.  But the RAS Plan name will vary.  This is the last // in the path name and must match when modifying DSS Files that contain different paths.
# Override as necessary if you are not using HMS-Commander to generate HMS Inputs
# If Override_DSS_Path = True, the last entry in the DSS path will be replaced with the HEC-RAS plan name (per HEC-RAS output naming convention)

if DSS_Search_Word == "HMS":
    Override_DSS_Path = False
else:
    Override_DSS_Path = True

# If HECRAS_project_folder is the same path as HECRAS_template_folder in "Build from DSS" mode (check if folder name is identical, not path), then raise an error
if Operation_Mode == "Build from DSS" and HECRAS_project_folder.split('\\')[-1] == HECRAS_template_folder.split('\\')[-1]:
    raise ValueError('HEC-RAS template folder and HEC-RAS project folder cannot have the same name in "Build from DSS" mode.  This will cause the folder to be deleted.  \nPlease note that Build from DSS overwrites your HEC-RAS Project Folder with a copy of the template.')




HEC-RAS Executable Path Used for batch file creation:  C:\Program Files (x86)\HEC\HEC-RAS\6.2\Ras.exe
Local Target Path Used for batch file creation:  C:\Temp_May_2D
The following file types are not needed for 1D compute runs and will be excluded:  []
Number of Parallel Threads for File Copy Operations:  4
Remote File Share Path:  C:


In [83]:
#3 Required Import Statements
# Auto-Install packages using subprocess (available in most base environments)
import sys
import subprocess 

packages = ["os", "shutil", "re", "pandas", "keyring", "getpass", "chardet", "datetime", "fnmatch", "threading", "time", "concurrent", "tkinter", "tqdm", "h5py", "numpy"]

# Logic to Install Packages
def install(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print("Installed " + package + " successfully")
    except subprocess.CalledProcessError:
        try:
            subprocess.check_call(["conda", "install", "-y", package])
        except subprocess.CalledProcessError as ex:
            print(f"Unable to install {package}: {str(ex)}")

for package in packages:
    try:
        # If the import succeeds, the package is installed, so we can move on
        __import__(package)
    except ImportError:
        # If the import fails, the package is not installed and we try to install it
        install(package)
from pathlib import Path
import sys
import os
import shutil
import pandas as pd
import re
import chardet
import datetime
import fnmatch
import getpass
import keyring
import threading
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from queue import Queue, Empty
from tqdm import tqdm
import urllib.request
import zipfile
import os
from urllib.error import URLError, HTTPError
import tempfile
import h5py
import numpy as np
import shlex
import tkinter as tk
from tkinter import ttk
from tkinter import simpledialog
from tkinter import filedialog, messagebox, ttk
import threading
import urllib.request

In [84]:
#4 --------  TKINTER GUI --------

# --------  Functions for TKINTER GUI --------

def confirm_deletion():
    # Function to handle deletion confirmation

    # Create a popup window
    popup = tk.Tk()
    popup.wm_title("Confirmation")

    # Set the popup to be always on top
    popup.attributes("-topmost", True)

    # Confirmation message
    message = f"You have Selected Build from DSS.  However, the output folder HECRAS_project_folder =" \
              "  {HECRAS_project_folder} " \
              "already exists.  Answering yes will delete all files in the HECRAS_project_folder folder. Are you sure?"
    label = tk.Label(popup, text=message, wraplength=300)
    label.pack(pady=10)

    # Function to handle the user's decision
    def on_confirm():
        popup.destroy()
        # Proceed with file deletion and copying template
        shutil.rmtree(HECRAS_project_folder)
        print(f"Copying contents from {HECRAS_template_folder} to {HECRAS_project_folder}")
        shutil.copytree(HECRAS_template_folder, HECRAS_project_folder)
        print("Finished Copying HEC-RAS Template")

    def on_cancel():
        popup.destroy()
        raise Exception("Build from DSS Mode Selected and Output Folder Exists, Check User Inputs")

    # Add buttons for user action
    yes_button = tk.Button(popup, text="Yes", command=on_confirm)
    yes_button.pack(side="left", padx=(20, 10), pady=20)

    no_button = tk.Button(popup, text="No", command=on_cancel)
    no_button.pack(side="right", padx=(10, 20), pady=20)

    # Start the GUI event loop
    popup.mainloop()




# Function to update the deploy targets list based on the checkbuttons' state
def update_deploy_targets():
    global HECRAS_Deploy_Targets
    HECRAS_Deploy_Targets = [target for target, var in deploy_target_vars.items() if var.get()]
    print("Updated HECRAS_Deploy_Targets:", HECRAS_Deploy_Targets)


def toggle_fields():
    if operation_mode.get() == "Run Missing":
        frame_build_dss.forget()  # Hide "Build from DSS" frame
        build_dss_note_label.pack_forget()  # Hide the "Build from DSS" note
        frame_run_missing.pack(fill='both', expand=True)  # Show "Run Missing" frame
        run_missing_note_label.pack(fill='x', pady=5)  # Show the "Run Missing" note
    elif operation_mode.get() == "Build from DSS":
        frame_run_missing.forget()  # Hide "Run Missing" frame
        run_missing_note_label.pack_forget()  # Hide the "Run Missing" note
        frame_build_dss.pack(fill='both', expand=True)  # Show "Build from DSS" frame
        build_dss_note_label.pack(fill='x', pady=5)  # Show the "Build from DSS" note
    else:
        # Default case if needed
        frame_run_missing.forget()
        frame_build_dss.forget()
        build_dss_note_label.pack_forget()  # Hide the "Build from DSS" note
        run_missing_note_label.pack_forget()  # Hide the "Run Missing" note





def on_infiltration_override_change(*args):
    if enable_infiltration_overrides.get():
        infiltration_from_rasmapper_csv_label.pack(anchor='w')
        infiltration_from_rasmapper_csv_entry.pack(fill='x', expand=True, anchor='w')
        browse_infiltration_csv_button.pack(side='right', padx=(5, 10))  # Show the browse button
        user_calibration_runs_csv_fullpath_label.pack(anchor='w')
        user_calibration_runs_csv_fullpath_entry.pack(fill='x', expand=True, anchor='w')
        browse_user_calibration_csv_button.pack(side='right', padx=(5, 10))  # Show the browse button

    else:
        infiltration_from_rasmapper_csv_label.pack_forget()
        infiltration_from_rasmapper_csv_entry.pack_forget()
        browse_infiltration_csv_button.pack_forget()  # Hide the browse button as well
        user_calibration_runs_csv_fullpath_label.pack_forget()
        user_calibration_runs_csv_fullpath_entry.pack_forget()
        browse_user_calibration_csv_button.pack_forget()  # Hide the browse button as well



# Function to open the Additional Settings window
def open_additional_settings():
    global additional_settings_window, actual_hecras_exe_path

    if 'additional_settings_window' in globals() and additional_settings_window.winfo_exists():
        additional_settings_window.lift()
        additional_settings_window.attributes("-topmost", True)
        return

    additional_settings_window = tk.Toplevel(root)
    additional_settings_window.title("Additional Settings")
    additional_settings_window.geometry("300x500")
    additional_settings_window.attributes("-topmost", True)  # Make window always on top
    additional_settings_window.grab_set()  # Make the window modal

    # Update tkinter variables with current global variable values
    number_parallel_runs.set(Number_Parallel_Runs)
    hecras_exe_path.set(actual_hecras_exe_path)
    psexec_run_in_system_account.set(Psexec_Run_In_System_Account)
    psexec_session_id.set(Psexec_Session_ID)
    psexec_priority.set(Psexec_Priority)
    remote_share_path.set(Remote_Share_Path)
    folder_safety_prefix_length.set(Folder_Safety_Prefix_Length)
    exclusions.set(','.join(exclusions) if isinstance(exclusions, list) else exclusions)
    file_copy_threads.set(File_Copy_Threads)
    remote_file_share_path.set(Remote_File_Share_Path)
    override_dss_path.set(Override_DSS_Path)

    # Creating a canvas and a scrollbar
    canvas = tk.Canvas(additional_settings_window)
    scrollbar = ttk.Scrollbar(additional_settings_window, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    # Configuring the canvas
    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )
    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    # Packing canvas and scrollbar
    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # Additional Settings Fields
    # Number of Parallel Runs
    number_parallel_runs_label = tk.Label(scrollable_frame, text="Number of Parallel Runs:")
    number_parallel_runs_label.pack(anchor='w')
    number_parallel_runs_entry = tk.Entry(scrollable_frame, textvariable=number_parallel_runs)
    number_parallel_runs_entry.pack(anchor='w')

    # HEC-RAS EXE Path
    hecras_exe_path_label = tk.Label(scrollable_frame, text="HEC-RAS EXE Path:")
    hecras_exe_path_label.pack(anchor='w')
    hecras_exe_path_entry = tk.Entry(scrollable_frame, textvariable=hecras_exe_path)
    hecras_exe_path_entry.pack(fill='x', expand=True, anchor='w')

    # Psexec Run In System Account
    psexec_run_in_system_account_label = tk.Label(scrollable_frame, text="Psexec Run In System Account:")
    psexec_run_in_system_account_label.pack(anchor='w')
    psexec_run_in_system_account_entry = ttk.Combobox(scrollable_frame, textvariable=psexec_run_in_system_account, values=["Yes", "No"])
    psexec_run_in_system_account_entry.pack(anchor='w')

    # Psexec Session ID
    psexec_session_id_label = tk.Label(scrollable_frame, text="Psexec Session ID:")
    psexec_session_id_label.pack(anchor='w')
    psexec_session_id_entry = tk.Entry(scrollable_frame, textvariable=psexec_session_id)
    psexec_session_id_entry.pack(anchor='w')

    # Psexec Priority
    psexec_priority_label = tk.Label(scrollable_frame, text="Psexec Priority:")
    psexec_priority_label.pack(anchor='w')
    psexec_priority_entry = ttk.Combobox(scrollable_frame, textvariable=psexec_priority, values=["low", "below normal", "normal"])
    psexec_priority_entry.pack(anchor='w')

    # Remote Share Path
    remote_share_path_label = tk.Label(scrollable_frame, text="Remote Share Path:")
    remote_share_path_label.pack(anchor='w')
    remote_share_path_entry = tk.Entry(scrollable_frame, textvariable=remote_share_path)
    remote_share_path_entry.pack(fill='x', expand=True, anchor='w')

    # Folder Safety Prefix Length
    folder_safety_prefix_length_label = tk.Label(scrollable_frame, text="Folder Safety Prefix Length:")
    folder_safety_prefix_length_label.pack(anchor='w')
    folder_safety_prefix_length_entry = tk.Entry(scrollable_frame, textvariable=folder_safety_prefix_length)
    folder_safety_prefix_length_entry.pack(anchor='w')

    # Exclusions
    exclusions_label = tk.Label(scrollable_frame, text="Exclusions:")
    exclusions_label.pack(anchor='w')
    exclusions_entry = tk.Entry(scrollable_frame, textvariable=exclusions)
    exclusions_entry.pack(anchor='w')

    # File Copy Threads
    file_copy_threads_label = tk.Label(scrollable_frame, text="File Copy Threads:")
    file_copy_threads_label.pack(anchor='w')
    file_copy_threads_entry = tk.Entry(scrollable_frame, textvariable=file_copy_threads)
    file_copy_threads_entry.pack(anchor='w')

    # Remote File Share Path
    remote_file_share_path_label = tk.Label(scrollable_frame, text="Remote File Share Path:")
    remote_file_share_path_label.pack(anchor='w')
    remote_file_share_path_entry = tk.Entry(scrollable_frame, textvariable=remote_file_share_path)
    remote_file_share_path_entry.pack(anchor='w')

    # Override DSS Path
    override_dss_path_checkbutton = tk.Checkbutton(scrollable_frame, text="Override DSS Path", variable=override_dss_path)
    override_dss_path_checkbutton.pack(anchor='w')

    # Close Button for Additional Settings Window
    close_button = tk.Button(scrollable_frame, text="Close", command=additional_settings_window.destroy)
    close_button.pack()

    # Ensure that the window is brought to the front
    additional_settings_window.lift()
    additional_settings_window.focus()


def toggle_additional_settings():
    if frame_additional_settings.winfo_ismapped():
        frame_additional_settings.forget()
    else:
        frame_additional_settings.pack(fill='both', expand=True)


    


def run_application():


    # [Existing code to update global variables]

    # Additional code to update global variables related to additional settings
    # E.g., global Number_Parallel_Runs
    # Number_Parallel_Runs = number_parallel_runs.get()

    # [Existing code to handle operation modes]


    # Apply the current settings before running the application logic
    # [Start of integrated apply_settings logic]
    global Operation_Mode, HECRAS_project_folder, HECRAS_Deploy_Targets, Number_Parallel_Runs
    global hecras_exe_path, Psexec_Run_In_System_Account, Psexec_Session_ID, Psexec_Priority
    global Remote_Share_Path, Folder_Safety_Prefix_Length, exclusions, File_Copy_Threads
    global Remote_File_Share_Path, Override_DSS_Path, HECRAS_template_folder, Plan_Number
    global DSS_Source_Folder, DSS_Search_Word, DSS_Replace_Word, DSS_File_Name_Filter_Word
    global Enable_Infiltration_Overrides, Infiltration_From_RASMapper_csv, user_calibration_runs_csv_fullpath
    # Also include Additional Settings
    global operation_mode, hecras_project_folder, hecras_deploy_targets, number_parallel_runs
    global hecras_exe_path, psexec_run_in_system_account, psexec_session_id, psexec_priority

    # Retrieve values from GUI and update the script variables
    Operation_Mode = operation_mode.get()
    HECRAS_project_folder = hecras_project_folder.get()
    HECRAS_Deploy_Targets = [target for target, var in deploy_target_vars.items() if var.get()]
    Number_Parallel_Runs = number_parallel_runs.get()
    hecras_exe_path = hecras_exe_path.get()
    Psexec_Run_In_System_Account = psexec_run_in_system_account.get()
    Psexec_Session_ID = psexec_session_id.get()
    Psexec_Priority = psexec_priority.get()
    Remote_Share_Path = remote_share_path.get()
    Folder_Safety_Prefix_Length = folder_safety_prefix_length.get()
    exclusions = exclusions.get().split(', ')
    File_Copy_Threads = file_copy_threads.get()
    Remote_File_Share_Path = remote_file_share_path.get()
    Override_DSS_Path = override_dss_path.get()
    HECRAS_template_folder = hecras_template_folder.get()
    Plan_Number = plan_number.get()
    DSS_Source_Folder = dss_source_folder.get()
    DSS_Search_Word = dss_search_word.get()
    DSS_Replace_Word = dss_replace_word.get()
    DSS_File_Name_Filter_Word = dss_file_name_filter_word.get()
    Enable_Infiltration_Overrides = enable_infiltration_overrides.get()
    Infiltration_From_RASMapper_csv = infiltration_from_rasmapper_csv.get()
    user_calibration_runs_csv_fullpath = user_calibration_runs_csv_fullpath.get()


    # Print statements for verification (can be removed or modified as needed)
    print("Settings Applied:")
    print("Operation Mode:", Operation_Mode)


    # Main application logic based on the operation mode
    if operation_mode.get() == "Run Missing":
        run_missing_mode()
    elif operation_mode.get() == "Build from DSS":
        build_from_dss_mode()

    # Close the GUI window after running the application logic
    root.destroy()



def run_missing_mode():
    # Placeholder for the 'Run Missing' mode logic
    # Here you can add the specific actions that should happen in this mode
    print("Running in 'Run Missing' mode")

    # Example of using the variables updated from the GUI
    print("HECRAS Project Folder:", HECRAS_project_folder)
    print("HECRAS Deploy Targets:", HECRAS_Deploy_Targets)
    
    # Additional Settings
    print("HEC-RAS EXE Path:", hecras_exe_path)
    print("Psexec Run In System Account:", Psexec_Run_In_System_Account)
    print("Psexec Session ID:", Psexec_Session_ID)
    print("Psexec Priority:", Psexec_Priority)
    print("Remote Share Path:", Remote_Share_Path)
    print("Folder Safety Prefix Length:", Folder_Safety_Prefix_Length)
    print("Exclusions:", exclusions)
    print("File Copy Threads:", File_Copy_Threads)
    print("Remote File Share Path:", Remote_File_Share_Path)
    print("Override DSS Path:", Override_DSS_Path)

    # This is where you would typically add code to execute the specific tasks for this mode,
    # such as processing data, running simulations, etc.
    # Example: run_hecras_simulation(HECRAS_project_folder, HECRAS_Deploy_Targets)
    # ...

    # Add your custom code for 'Run Missing' mode here.
    # Nothing for now, the script just executes after the GUI is closed


def build_from_dss_mode():
    # Placeholder for the 'Build from DSS' mode logic
    # Here you can add the specific actions that should happen in this mode
    print("Building from DSS")

    # Example of using the variables updated from the GUI
    print("HECRAS Template Folder:", HECRAS_template_folder)
    print("Plan Number:", Plan_Number)
    print("DSS Source Folder:", DSS_Source_Folder)
    print("DSS Search Word:", DSS_Search_Word)
    print("DSS Replace Word:", DSS_Replace_Word)
    print("DSS File Name Filter Word:", DSS_File_Name_Filter_Word)
    print("Enable Infiltration Overrides:", Enable_Infiltration_Overrides)
    print("Infiltration From RASMapper CSV:", Infiltration_From_RASMapper_csv)
    print("User Calibration Runs CSV Fullpath:", user_calibration_runs_csv_fullpath)
    
    # Additional Settings
    print("HEC-RAS EXE Path:", hecras_exe_path)
    print("Psexec Run In System Account:", Psexec_Run_In_System_Account)
    print("Psexec Session ID:", Psexec_Session_ID)
    print("Psexec Priority:", Psexec_Priority)
    print("Remote Share Path:", Remote_Share_Path)
    print("Folder Safety Prefix Length:", Folder_Safety_Prefix_Length)
    print("Exclusions:", exclusions)
    print("File Copy Threads:", File_Copy_Threads)
    print("Remote File Share Path:", Remote_File_Share_Path)
    print("Override DSS Path:", Override_DSS_Path)



    # This is where you would typically add code to execute the specific tasks for this mode,
    # such as building new HEC-RAS plans, processing data, etc.
    # Example: create_hecras_plan(HECRAS_template_folder, Plan_Number)
    # ...

    # Add your custom code for 'Build from DSS' mode here.

    # Nothing for now, the script just executes after the GUI is closed



def close_and_raise_exception():
    """
    Function to close the Tkinter window and raise an exception.
    """
    root.quit()  # Stop the main loop
    root.destroy()  # Destroy the root window
    raise Exception("Application closed by user")  # Raise an exception with a custom message



def on_close():
    global close_exception_flag
    close_exception_flag = True  # Set the flag
    root.quit()  # Stop the main loop
    root.destroy()  # Close the window



def initialize_gui():
    toggle_fields()
    frame_additional_settings.forget()



def browse_folder(variable, default_value):
    folder_selected = filedialog.askdirectory()
    if folder_selected:
        variable.set(folder_selected)
    else:
        variable.set(default_value)

def browse_dss_source_folder():
    folder_selected = filedialog.askdirectory()
    if folder_selected:
        dss_source_folder.set(folder_selected)

def browse_csv_file(variable, default_value):
    filename = filedialog.askopenfilename(
        filetypes=[("CSV files", "*.csv")],
        defaultextension="*.csv"
    )
    if filename:
        variable.set(filename)
    else:
        # Only update the variable if a new file is selected
        # If the user cancels, keep the current value
        if not variable.get():
            variable.set(default_value)


 

# -------- TKINTER GUI --------




# Initialize Main Window
root = tk.Tk()
root.title("RAS-Commander 1.0")
root.geometry("900x900")

# Bind the function to the close event (raises exception if window closed)
root.protocol("WM_DELETE_WINDOW", on_close)

# Set the window to be always on top
root.attributes('-topmost', True)

# Dictionary to hold the BooleanVar objects for each deploy target
deploy_target_vars = {}

# Create a frame to hold the checkbuttons in a grid
deploy_targets_frame = tk.LabelFrame(root, text="HECRAS Deploy-Execute Target Folders")
deploy_targets_frame.pack(fill="both", expand=True, padx=10, pady=10)

# Set grid column configuration for equal width
deploy_targets_frame.grid_columnconfigure(0, weight=1)
deploy_targets_frame.grid_columnconfigure(1, weight=1)

# Dictionary to hold the BooleanVar objects for each deploy target
deploy_target_vars = {}

# Variables to keep track of grid position
row_index = 0
column_index = 0

# Create a checkbutton for each deploy target in a grid layout
for i, target in enumerate(HECRAS_Deploy_Targets):
    var = tk.BooleanVar(value=True)  # Default to checked
    deploy_target_vars[target] = var
    checkbutton = tk.Checkbutton(deploy_targets_frame, text=target, variable=var)
    checkbutton.grid(row=row_index, column=column_index, sticky="w", padx=5, pady=2)

    # Alternate between column 0 and 1
    column_index = 1 - column_index

    # Increment row index every two check buttons
    if i % 2 != 0:
        row_index += 1




# Create a frame for additional settings and initially hide it
frame_additional_settings = tk.Frame(root)
frame_additional_settings.pack(fill='both', expand=True)
frame_additional_settings.forget()  # Hide the frame initially



# Set the initial values of tkinter variables from the script variables
root.setvar(name='operation_mode', value=Operation_Mode)
root.setvar(name='hecras_project_folder', value=HECRAS_project_folder)
root.setvar(name='hecras_deploy_targets', value=str(HECRAS_Deploy_Targets))
root.setvar(name='number_parallel_runs', value=Number_Parallel_Runs)
root.setvar(name='hecras_exe_path', value=hecras_exe_path)
root.setvar(name='psexec_run_in_system_account', value=Psexec_Run_In_System_Account)
root.setvar(name='psexec_session_id', value=Psexec_Session_ID)
root.setvar(name='psexec_priority', value=Psexec_Priority)
root.setvar(name='remote_share_path', value=Remote_Share_Path)
root.setvar(name='folder_safety_prefix_length', value=Folder_Safety_Prefix_Length)
# Check if 'exclusions' is a list and join it, else use it as it is
exclusions_value = ','.join(exclusions) if isinstance(exclusions, list) else exclusions
root.setvar(name='exclusions', value=exclusions_value)  # Assuming exclusions is a list
root.setvar(name='file_copy_threads', value=File_Copy_Threads)
root.setvar(name='remote_file_share_path', value=Remote_File_Share_Path)
root.setvar(name='override_dss_path', value=Override_DSS_Path)

root.setvar(name='hecras_template_folder', value=HECRAS_template_folder)
root.setvar(name='plan_number', value=Plan_Number)
root.setvar(name='dss_source_folder', value=DSS_Source_Folder)
root.setvar(name='dss_search_word', value=DSS_Search_Word)
root.setvar(name='dss_replace_word', value=DSS_Replace_Word)
root.setvar(name='dss_file_name_filter_word', value=DSS_File_Name_Filter_Word)
root.setvar(name='enable_infiltration_overrides', value=Enable_Infiltration_Overrides)
root.setvar(name='infiltration_from_rasmapper_csv', value=Infiltration_From_RASMapper_csv)
root.setvar(name='user_calibration_runs_csv_fullpath', value=user_calibration_runs_csv_fullpath)

# Global tkinter variables for data binding
operation_mode = tk.StringVar(root, name='operation_mode')
hecras_project_folder = tk.StringVar(root, name='hecras_project_folder')
hecras_deploy_targets = tk.StringVar(root, name='hecras_deploy_targets')
number_parallel_runs = tk.IntVar(root, name='number_parallel_runs')

# Define the actual executable path
actual_hecras_exe_path = hecras_exe_path

# Initialize the StringVar with the actual path
hecras_exe_path = tk.StringVar(root, name='hecras_exe_path', value=actual_hecras_exe_path)
psexec_run_in_system_account = tk.StringVar(root, name='psexec_run_in_system_account')
psexec_session_id = tk.IntVar(root, name='psexec_session_id')
psexec_priority = tk.StringVar(root, name='psexec_priority')
remote_share_path = tk.StringVar(root, name='remote_share_path')
folder_safety_prefix_length = tk.IntVar(root, name='folder_safety_prefix_length')
exclusions = tk.StringVar(root, name='exclusions')
file_copy_threads = tk.IntVar(root, name='file_copy_threads')
remote_file_share_path = tk.StringVar(root, name='remote_file_share_path')
override_dss_path = tk.BooleanVar(root, name='override_dss_path')

# Specific to "Build from DSS"
hecras_template_folder = tk.StringVar(root, name='hecras_template_folder')
plan_number = tk.StringVar(root, name='plan_number')
dss_source_folder = tk.StringVar(root, name='dss_source_folder')
dss_search_word = tk.StringVar(root, name='dss_search_word')
dss_replace_word = tk.StringVar(root, name='dss_replace_word')
dss_file_name_filter_word = tk.StringVar(root, name='dss_file_name_filter_word')
enable_infiltration_overrides = tk.BooleanVar(root, name='enable_infiltration_overrides')
infiltration_from_rasmapper_csv = tk.StringVar(root, name='infiltration_from_rasmapper_csv')
user_calibration_runs_csv_fullpath = tk.StringVar(root, name='user_calibration_runs_csv_fullpath')





# Define a larger font for radio buttons
large_font = ('Verdana', 12)

# Create the bold font style
bold_font = ('Verdana', 10, 'bold')

# Create the note label for "Build from DSS" mode
build_dss_note_label = tk.Label(root, text="In Build From DSS Mode, the HECRAS Project Folder will be overwritten", font=bold_font)
# Don't pack the label yet; it will be handled by the toggle_fields function

# Create the note label for "Run Missing" mode
run_missing_note_label = tk.Label(root, text="In Run Missing Mode, any plans without DSS outputs will be run.\nFor existing projects and retrying failed runs", fg='green')

# Frame for operation modes
operation_mode_frame = tk.Frame(root)
operation_mode_frame.pack(fill='x', padx=10, pady=5)

operation_mode_label = tk.Label(operation_mode_frame, text="Select Operation Mode:", font=large_font)
operation_mode_label.pack(side=tk.TOP, pady=(10, 0))

operation_mode_radio_run_missing = tk.Radiobutton(operation_mode_frame, text="Run Missing", variable=operation_mode, value="Run Missing", font=large_font)
operation_mode_radio_run_missing.pack(side=tk.LEFT, expand=True)

operation_mode_radio_build_dss = tk.Radiobutton(operation_mode_frame, text="Build from DSS", variable=operation_mode, value="Build from DSS", font=large_font)
operation_mode_radio_build_dss.pack(side=tk.LEFT, expand=True)

# Button to open the Additional Settings window
additional_settings_button = tk.Button(root, text="Additional Settings", command=open_additional_settings)

# Adjust the height of the button (e.g., setting height to 2)
# This might not work as expected on all platforms due to theme and OS differences
additional_settings_button.config(height=2)

# Alternatively, increase the size using padding
# Here, 20 pixels padding is added at the top and bottom
additional_settings_button.pack(fill='x', padx=10, pady=20)

# If the first approach doesn't work as expected, you can combine both methods.

# Define the frames for each operation mode, but don't pack them yet
frame_run_missing = tk.Frame(root)
frame_build_dss = tk.Frame(root)

# Call toggle_fields to initialize the correct display based on the default or previously set operation mode
toggle_fields()




# Fields specific to "Run Missing"
#frame_run_missing = tk.Frame(root)
#hecras_project_folder_label = tk.Label(frame_run_missing, text="HECRAS Project Folder2:")
#hecras_project_folder_label.pack(anchor='w')


#hecras_project_folder_entry_frame = tk.Frame(frame_run_missing)
#hecras_project_folder_entry_frame.pack(fill='x', expand=True, anchor='w')
#hecras_project_folder_entry = tk.Entry(hecras_project_folder_entry_frame, textvariable=hecras_project_folder)
#hecras_project_folder_entry.pack(side='left', fill='x', expand=True)
#browse_button = tk.Button(hecras_project_folder_entry_frame, text="Browse", command=lambda: browse_folder(hecras_project_folder))
#browse_button.pack(side='right')


frame_build_dss = tk.Frame(root)

# Fields specific to "Build from DSS"

# Assuming default_hecras_template_folder is the default value for the HECRAS template folder
default_hecras_template_folder = HECRAS_template_folder  # Set this to your default path

# HECRAS Template Folder Entry
hecras_template_folder_frame = tk.Frame(frame_build_dss)
hecras_template_folder_frame.pack(fill='x', expand=True, padx=10, anchor='w')
hecras_template_folder_label = tk.Label(hecras_template_folder_frame, text="HECRAS Template Folder:")
hecras_template_folder_label.pack(side='left', anchor='w', padx=(10, 5))
hecras_template_folder_entry = tk.Entry(hecras_template_folder_frame, textvariable=hecras_template_folder)
hecras_template_folder_entry.pack(side='left', fill='x', expand=True, padx=(5, 5), anchor='w')
browse_template_button = tk.Button(hecras_template_folder_frame, text="Browse", command=lambda: browse_folder(hecras_template_folder, default_hecras_template_folder))
browse_template_button.pack(side='right', padx=(5, 10))

# Plan Number Entry
plan_number_frame = tk.Frame(frame_build_dss)
plan_number_frame.pack(fill='x', expand=True, padx=10, anchor='w')
plan_number_label = tk.Label(plan_number_frame, text="Plan Number:")
plan_number_label.pack(side='left', anchor='w', padx=(10, 5))
plan_number_entry = tk.Entry(plan_number_frame, textvariable=plan_number)
plan_number_entry.pack(side='left', padx=(5, 5), anchor='w')

# DSS Source Folder Entry
dss_source_folder_frame = tk.Frame(frame_build_dss)
dss_source_folder_frame.pack(fill='x', expand=True, padx=10, anchor='w')
dss_source_folder_label = tk.Label(dss_source_folder_frame, text="DSS Source Folder:")
dss_source_folder_label.pack(side='left', anchor='w', padx=(10, 5))
dss_source_folder_entry = tk.Entry(dss_source_folder_frame, textvariable=dss_source_folder)
dss_source_folder_entry.pack(side='left', fill='x', expand=True, padx=(5, 5), anchor='w')
browse_dss_source_button = tk.Button(dss_source_folder_frame, text="Browse", command=browse_dss_source_folder)
browse_dss_source_button.pack(side='right', padx=(5, 10))

# DSS Search Word Entry
dss_search_word_frame = tk.Frame(frame_build_dss)
dss_search_word_frame.pack(fill='x', expand=True, padx=10, anchor='w')
dss_search_word_label = tk.Label(dss_search_word_frame, text="DSS Search Word:")
dss_search_word_label.pack(side='left', anchor='w', padx=(10, 5))
dss_search_word_entry = tk.Entry(dss_search_word_frame, textvariable=dss_search_word)
dss_search_word_entry.pack(side='left', padx=(5, 5), anchor='w')


# DSS Replace Word Entry
dss_replace_word_frame = tk.Frame(frame_build_dss)
dss_replace_word_frame.pack(fill='x', expand=True, padx=10, anchor='w')
dss_replace_word_label = tk.Label(dss_replace_word_frame, text="DSS Replace Word:")
dss_replace_word_label.pack(side='left', anchor='w', padx=(10, 5))
dss_replace_word_entry = tk.Entry(dss_replace_word_frame, textvariable=dss_replace_word)
dss_replace_word_entry.pack(side='left', padx=(5, 5), anchor='w')

# DSS File Name Filter Word Entry
dss_file_name_filter_word_frame = tk.Frame(frame_build_dss)
dss_file_name_filter_word_frame.pack(fill='x', expand=True, padx=10, anchor='w')
dss_file_name_filter_word_label = tk.Label(dss_file_name_filter_word_frame, text="DSS File Name Filter Word:")
dss_file_name_filter_word_label.pack(side='left', padx=(10, 5), anchor='w')
dss_file_name_filter_word_entry = tk.Entry(dss_file_name_filter_word_frame, textvariable=dss_file_name_filter_word)
dss_file_name_filter_word_entry.pack(side='left', padx=(5, 5), anchor='w')

# Infiltration Override Checkbutton
enable_infiltration_overrides_frame = tk.Frame(frame_build_dss)
enable_infiltration_overrides_frame.pack(fill='x', expand=True, padx=20,pady=20, anchor='w')
enable_infiltration_overrides_checkbutton = tk.Checkbutton(enable_infiltration_overrides_frame, text="Enable Infiltration Overrides", variable=enable_infiltration_overrides, command=on_infiltration_override_change)
enable_infiltration_overrides_checkbutton.pack(anchor='w')

# Infiltration From RASMapper CSV Entry
infiltration_from_rasmapper_csv_frame = tk.Frame(frame_build_dss)
infiltration_from_rasmapper_csv_frame.pack(fill='x', expand=True, padx=10, anchor='w')
infiltration_from_rasmapper_csv_label = tk.Label(infiltration_from_rasmapper_csv_frame, text="Infiltration From RASMapper CSV:")
infiltration_from_rasmapper_csv_label.pack(side='left', padx=(10, 5), anchor='w')
infiltration_from_rasmapper_csv_entry = tk.Entry(infiltration_from_rasmapper_csv_frame, textvariable=infiltration_from_rasmapper_csv)
infiltration_from_rasmapper_csv_entry.pack(side='left', fill='x', expand=True, padx=(5, 5), anchor='w')
browse_infiltration_csv_button = tk.Button(infiltration_from_rasmapper_csv_frame, text="Browse", command=lambda: browse_csv_file(infiltration_from_rasmapper_csv, infiltration_from_rasmapper_csv.get()))
browse_infiltration_csv_button.pack(side='right', padx=(5, 10))

# User Calibration Runs CSV Fullpath Entry
user_calibration_runs_csv_fullpath_frame = tk.Frame(frame_build_dss)
user_calibration_runs_csv_fullpath_frame.pack(fill='x', expand=True, padx=10, anchor='w')
user_calibration_runs_csv_fullpath_label = tk.Label(user_calibration_runs_csv_fullpath_frame, text="User Calibration Runs CSV Fullpath:")
user_calibration_runs_csv_fullpath_label.pack(side='left', padx=(10, 5), anchor='w')
user_calibration_runs_csv_fullpath_entry = tk.Entry(user_calibration_runs_csv_fullpath_frame, textvariable=user_calibration_runs_csv_fullpath)
user_calibration_runs_csv_fullpath_entry.pack(side='left', fill='x', expand=True, padx=(5, 5), anchor='w')
browse_user_calibration_csv_button = tk.Button(user_calibration_runs_csv_fullpath_frame, text="Browse", command=lambda: browse_csv_file(user_calibration_runs_csv_fullpath, user_calibration_runs_csv_fullpath.get()))
browse_user_calibration_csv_button.pack(side='right', padx=(5, 10))


# Call the on_infiltration_override_change function to initialize the display state of infiltration override fields
on_infiltration_override_change()



# Initially, don't pack this frame. It will be managed by the toggle_fields function in part 3.




# Button to open the Additional Settings window
#collapse_button = tk.Button(root, text="Additional Settings", command=open_additional_settings)
#collapse_button.pack()

#frame_additional_settings = tk.Frame(root)

# Additional Settings Fields
number_parallel_runs_label = tk.Label(frame_additional_settings, text="Number of Parallel Runs:")
number_parallel_runs_label.pack(anchor='w')
number_parallel_runs_entry = tk.Entry(frame_additional_settings, textvariable=number_parallel_runs)
number_parallel_runs_entry.pack(anchor='w')

hecras_exe_path_label = tk.Label(frame_additional_settings, text="HEC-RAS EXE Path:")
hecras_exe_path_label.pack(anchor='w')
hecras_exe_path_entry = tk.Entry(frame_additional_settings, textvariable=hecras_exe_path)

hecras_exe_path_entry.pack(fill='x', expand=True, anchor='w')

psexec_run_in_system_account_label = tk.Label(frame_additional_settings, text="Psexec Run In System Account:")
psexec_run_in_system_account_label.pack(anchor='w')
psexec_run_in_system_account_entry = ttk.Combobox(frame_additional_settings, textvariable=psexec_run_in_system_account, values=["Yes", "No"])
psexec_run_in_system_account_entry.pack(anchor='w')

psexec_session_id_label = tk.Label(frame_additional_settings, text="Psexec Session ID:")
psexec_session_id_label.pack(anchor='w')
psexec_session_id_entry = tk.Entry(frame_additional_settings, textvariable=psexec_session_id)
psexec_session_id_entry.pack(anchor='w')

psexec_priority_label = tk.Label(frame_additional_settings, text="Psexec Priority:")
psexec_priority_label.pack(anchor='w')
psexec_priority_entry = ttk.Combobox(frame_additional_settings, textvariable=psexec_priority, values=["low", "below normal", "normal"])
psexec_priority_entry.pack(anchor='w')

remote_share_path_label = tk.Label(frame_additional_settings, text="Remote Share Path:")
remote_share_path_label.pack(anchor='w')
remote_share_path_entry = tk.Entry(frame_additional_settings, textvariable=remote_share_path)
remote_share_path_entry.pack(fill='x', expand=True, anchor='w')

folder_safety_prefix_length_label = tk.Label(frame_additional_settings, text="Folder Safety Prefix Length:")
folder_safety_prefix_length_label.pack(anchor='w')
folder_safety_prefix_length_entry = tk.Entry(frame_additional_settings, textvariable=folder_safety_prefix_length)
folder_safety_prefix_length_entry.pack(anchor='w')

exclusions_label = tk.Label(frame_additional_settings, text="Exclusions:")
exclusions_label.pack(anchor='w')
exclusions_entry = tk.Entry(frame_additional_settings, textvariable=exclusions)
exclusions_entry.pack(anchor='w')

file_copy_threads_label = tk.Label(frame_additional_settings, text="File Copy Threads:")
file_copy_threads_label.pack(anchor='w')
file_copy_threads_entry = tk.Entry(frame_additional_settings, textvariable=file_copy_threads)
file_copy_threads_entry.pack(anchor='w')

remote_file_share_path_label = tk.Label(frame_additional_settings, text="Remote File Share Path:")
remote_file_share_path_label.pack(anchor='w')
remote_file_share_path_entry = tk.Entry(frame_additional_settings, textvariable=remote_file_share_path)
remote_file_share_path_entry.pack(anchor='w')

override_dss_path_checkbutton = tk.Checkbutton(frame_additional_settings, text="Override DSS Path", variable=override_dss_path)
override_dss_path_checkbutton.pack(anchor='w')

# Initially hide the additional settings
frame_additional_settings.forget()


default_hecras_project_folder = HECRAS_project_folder  # Set this to your default path


# Frame for HECRAS Project Folder
hecras_project_folder_frame = tk.Frame(root)
hecras_project_folder_frame.pack(fill='x', padx=15, pady=5, anchor='w')


hecras_project_folder_label = tk.Label(hecras_project_folder_frame, text="HECRAS Project Folder:")
hecras_project_folder_label.pack(side='left', padx=5, anchor='w')

hecras_project_folder_entry_frame = tk.Frame(hecras_project_folder_frame)
hecras_project_folder_entry_frame.pack(fill='x', expand=True, pady=5, anchor='w')

hecras_project_folder_entry = tk.Entry(hecras_project_folder_entry_frame, textvariable=hecras_project_folder)
hecras_project_folder_entry.pack(side='left', fill='x', expand=True, padx=5, anchor='w')

browse_button = tk.Button(hecras_project_folder_entry_frame, text="Browse", command=lambda: browse_folder(hecras_project_folder, default_hecras_project_folder))
browse_button.pack(side=tk.RIGHT, padx=5)

# Now create and pack the note label
build_dss_note_label = tk.Label(root, text="In Build From DSS Mode, the HECRAS Project Folder will be overwritten", font=bold_font, fg='red')
#run_missing_note_label = tk.Label(root, text="In Run Missing Mode, any plans without DSS outputs will be run.\nFor existing projects and retrying failed runs", fg='green')


# Separator before the action buttons
separator = tk.ttk.Separator(root, orient='horizontal')
separator.pack(fill='x', padx=5, pady=5)

# Run and Cancel buttons frame
buttons_frame = tk.Frame(root)
buttons_frame.pack(side=tk.BOTTOM, fill='x', padx=10, pady=5)

run_button = tk.Button(buttons_frame, text="Run HEC-RAS\nIn Parallel", command=run_application, bg="#90EE90")
run_button.pack(side=tk.LEFT, padx=10, pady=10)
run_button.config(height=5, width=45)

cancel_button = tk.Button(buttons_frame, text="Cancel/Exit", command=close_and_raise_exception, bg="#FFB6C1")
cancel_button.pack(side=tk.RIGHT, padx=10, pady=10)
cancel_button.config(height=5, width=45)

# Bind the toggle_fields function to the Radiobuttons
operation_mode_radio_run_missing.configure(command=toggle_fields)
operation_mode_radio_build_dss.configure(command=toggle_fields)



# Initialize the frames and widgets for 'Run Missing' and 'Build from DSS' modes
initialize_gui()

# Global flag to indicate whether to raise an exception after closing the window
close_exception_flag = False

# Bind the function to the window's close event
root.protocol("WM_DELETE_WINDOW", on_close)

# Run the main application loop
root.mainloop()

# After exiting the main loop, check the flag and raise an exception if set
if close_exception_flag:
    raise Exception("Window closed by user without running")





Exception: Window closed by user without running

In [ ]:
#4 Check for psexec64.exe, get password, and test connection to remote machines

# IF YOU ENTER THE WRONG PASSWORD, CHANGE THE SERVICE NAME AND TRY AGAIN
# IF YOU ENTER THE WRONG USERNAME, DELETE THE USERNAME.TXT FILE AND TRY AGAIN

# General note: when trying to execute remote shell commands with psexec, pass shell=True to subprocess.run() to avoid errors

# Specify the psexec path
psexec_path = r"C:\psexec\PsExec64.exe" 
print("Target PSEXEC Path:", psexec_path)

# Constants
USERNAME_FILE = "username.txt"
SERVICE_NAME = "remote_machine_credential"

# Get the temp directory path
temp_directory = tempfile.gettempdir()

# Specify the psexec path within this directory
psexec_path = os.path.join(temp_directory, "psexec", "PsExec64.exe")
print("Target PSEXEC Path:", psexec_path)

def download_and_extract_pstools(psexec_path):
    # Check if the directory exists
    dir_path = os.path.dirname(psexec_path)
    if not os.path.exists(dir_path):
        print(f"Directory {dir_path} does not exist.")
    else:
        # Check if PsExec64.exe already exists (case-insensitive)
        if any(os.path.exists(os.path.join(dir_path, f)) for f in os.listdir(dir_path) if f.lower() == 'psexec64.exe'):
            print(f"The file PsExec64.exe exists at {dir_path}.")
            return
    
    print(f"PsExec64.exe not found at {dir_path}. Proceeding to download and extract PSTools.")
    
    # Define the download URL
    download_url = 'https://download.sysinternals.com/files/PSTools.zip'
    
    # Specify the directory where all tools will be extracted
    destination_dir = dir_path
    
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        print(f"Created directory {destination_dir}.")
    
    # Define the path where the ZIP file will be saved
    zip_file_path = os.path.join(destination_dir, 'PSTools.zip')
    
    try:
        # Download the ZIP file
        urllib.request.urlretrieve(download_url, zip_file_path)
        print(f"Downloaded PSTools.zip to {zip_file_path}")
        
        # Extract all the contents of the ZIP file
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(destination_dir)
            print(f"Extracted all contents of PSTools.zip to {destination_dir}")
    
    except (HTTPError, URLError) as e:
        print(f"[ERROR] Could not download PSTools.zip due to {e}.")
        print("Please download PsExec manually from https://learn.microsoft.com/en-us/sysinternals/downloads/psexec")

# Specify the psexec path
psexec_path = r"C:\psexec\PsExec64.exe"
print("Target PSEXEC Path:", psexec_path)

# Download and extract PSTools if PsExec64.exe doesn't exist
download_and_extract_pstools(psexec_path)

# Utilize Keyring to request and validate User Credentials for Remote Execution
# This should be your FULL Microsoft Windows user login (with domain or local machine name: DOMAIN\USER or MACHINE\USER)


'''
Debug:  Clear all user credentials

if os.path.exists(USERNAME_FILE):
    os.remove(USERNAME_FILE)

'''
def load_or_prompt_credentials():
    # Load or prompt for username
    if os.path.exists(USERNAME_FILE):
        with open(USERNAME_FILE, 'r') as f:
            username = f.read().strip()
            print(f"Loaded username {username} from {USERNAME_FILE}")
    else:
        root = tk.Tk()
        root.withdraw()  # Hide the main window (optional)
        root.lift()  # Bring the dialog to the front
        root.attributes('-topmost', True)  # Make sure the window appears on top
        username = simpledialog.askstring("Credentials", "Enter the username (format: DOMAIN\\USER):", parent=root)
        if username:
            with open(USERNAME_FILE, 'w') as f:
                f.write(username)
        else:
            raise Exception("Username is required to proceed.")
        root.destroy()

    # Load or prompt for password
    password = keyring.get_password(SERVICE_NAME, username)
    if password is None:
        root = tk.Tk()
        root.withdraw()  # Hide the main window (optional)
        root.lift()  # Bring the dialog to the front
        root.attributes('-topmost', True)  # Make sure the window appears on top
        password = simpledialog.askstring("Credentials", "Enter your password:", parent=root, show="*")
        if password:
            keyring.set_password(SERVICE_NAME, username, password)
        else:
            raise Exception("Password is required to proceed.")
        root.destroy()
    else:
        pass  # Placeholder to maintain structure; can be removed

    return username, password

# Initially load the credentials
username, password = load_or_prompt_credentials()
print("Initial credentials loaded.")


# NOTE this is only medium security, and prevents you from exposing your username by sharing the script.  
# Usernames are stored in clear text, revise if greater security is desired

'''

### Function to Test PsExec Connection

This function aims to test the PsExec connection to a remote machine. If the first attempt to connect fails, it will prompt the user for credentials and try again.

#### Parameters:

- `psexec_path`: The path where PsExec is located.
- `remote_machine`: The name or IP address of the remote machine you want to connect to.
- `username`: The username for accessing the remote machine.
- `password`: The password for accessing the remote machine.

#### Returns:

- Returns `True` if the connection is successful, otherwise returns `False`.


            obfuscated_password = "********"
        
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
            # Print the command with the obfuscated password by replacing it only for the print statement
            print(f"[{timestamp}] Executing command: {psexec_cmd.replace(password, obfuscated_password)}")


'''
obfuscated_password = "********"

# Step 3: Decouple running command logic into a separate function
def run_psexec_command(remote_machine, username, password, command):
    testrun_psexec_command = f'powershell -WindowStyle Hidden -Command "& \'{psexec_path}\' \\\\{remote_machine} -u {username} -p {password} -i {Psexec_Session_ID} -accepteula {command}"'
    print(f"Executing command: {testrun_psexec_command.replace(password, obfuscated_password)}")
    
    try:
        subprocess.run(testrun_psexec_command, check=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Command execution failed with error: {str(e).replace(password, obfuscated_password)}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {str(e).replace(password, obfuscated_password)}")
        return False

# Function to test PsExec connection
def test_psexec_connection(remote_machine, username, password):
    
    
    # Initial Test
    if run_psexec_command(remote_machine, username, password, 'ipconfig'):
        print(f"Successfully connected to {remote_machine}")
        print("")
        return True
    
    # Step 5: Re-attempt logic
    for _ in range(2):  # Number of retries can be adjusted
        print(f"Failed to connect to {remote_machine}. Retrying...")
        
        # Delete Credentials and Re-prompt for both username and password
        keyring.delete_password(SERVICE_NAME, username)

        if os.path.exists(USERNAME_FILE):
            os.remove(USERNAME_FILE)
                        
        username, password = load_or_prompt_credentials()
        
        if run_psexec_command(remote_machine, username, password, 'dir'):
            print(f"Successfully connected to {remote_machine} on retry")
            return True
        else:
            print(f"Failed to connect to {remote_machine} on retry - Check Password or Permissions and Try Again")
            # Add failed machines to a list for later
            failed_machines.append(remote_machine)
        
    
    print(f"Retry failed for {remote_machine}. Please check your credentials or network connection.")
    return False

# Initialize list of failed machines
failed_machines = []
# Test the connection to each remote machine
for folder in HECRAS_Deploy_Targets:
    if not folder.startswith("\\\\"):  # Check if folder does not start with \\
        continue  # Skip the rest of the loop for this folder
    
    remote_machine = folder.split('\\', 3)[2]
    print("remote_machine: ", remote_machine)
    print("")

    psexec_test_result = test_psexec_connection(remote_machine, username, password)



In [ ]:
#5 Populate RAS Project Name and Other Paths 

# Check Operation Mode and generate error if invalid mode is entered
valid_modes = ["Build from DSS", "Run Missing"]

if Operation_Mode not in valid_modes:
    raise ValueError(f"Invalid Operation_Mode: {Operation_Mode}. Please choose from {valid_modes}")


def get_HECRAS_project_name(folder_path):
    """Return the HEC-RAS project name derived from the folder containing .rasmap files."""
    for file_name in os.listdir(folder_path):
        # Check if the file has .rasmap extension
        if file_name.endswith('.rasmap'):
            # Return the file name without the .rasmap extension
            return file_name[:-7]  
    # Return None if no .rasmap file is found
    return None

def get_Folder_Safety_Prefix(folder_path, prefix_length):
    """Return the Folder Safety Prefix derived from the folder name."""
    folder_name = folder_path.split("\\")[-1]
    return folder_name[:prefix_length]

# Derive the HEC-RAS Project Name from the HEC-RAS Folder
HECRAS_project_name = get_HECRAS_project_name(HECRAS_template_folder)
print("HEC-RAS Project Name:", HECRAS_project_name)

# HEC-RAS PRJ File is the project name with a .prj extension
HECRAS_prj_file = HECRAS_project_name + ".prj"
print("HEC-RAS PRJ File:", HECRAS_prj_file)

# Set Output CSV with Missing DSS files and Extension Numbers
plan_title_csv_file_name = "RAS-Cmdr New Plans and Titles.csv"
print("Plan Title Output CSV:", plan_title_csv_file_name)

Remote_Base_Directory = Remote_Share_Path + HECRAS_Deploy_Targets[0].split('\\', 3)[-1]
print("Local Target Path Used for batch file creation:", Remote_Base_Directory) 

# Determine Folder_Safety_Prefix based on Folder_Safety_Prefix_Length
Folder_Safety_Prefix = get_Folder_Safety_Prefix(HECRAS_project_folder, Folder_Safety_Prefix_Length)
print("Folder Safety Prefix:", Folder_Safety_Prefix)

# Check if the RAS project folder exists
if not os.path.exists(HECRAS_project_folder):
    # If not, create it
    print(f"Creating directory {HECRAS_project_folder}")
    os.makedirs(HECRAS_project_folder)



In [ ]:
#6 If Build from DSS, copy template folder to output folder and copy DSS files from DSS_Source_Folder and Read Read Existing PRJ and extract .pXX and .uXX file mapping

# If Operation_Mode is "Build from DSS", perform the copying and deletion if necessary
if Operation_Mode == "Build from DSS":
    if os.path.isdir(HECRAS_project_folder) and os.listdir(HECRAS_project_folder):
        # Prompt the user for confirmation before deleting files and copying the template
        confirm_deletion()
    else:
        raise Exception("Build from DSS Mode Selected and Output Folder Exists, Check User Inputs")
elif Operation_Mode == "Run Missing":
    print(f"Run Missing Mode is selected.  HECRAS_project_folder already exists and will not be modified")
else:
    # This else aligns with the first if to handle invalid operation modes
    # Copy the template as the folder is either newly created or empty
    print(f"Copying contents from {HECRAS_template_folder} to {HECRAS_project_folder}")
    shutil.copytree(HECRAS_template_folder, HECRAS_project_folder, dirs_exist_ok=True)
    print("Finished Copying HEC-RAS Template")
    raise ValueError(f"Invalid Operation_Mode: {Operation_Mode}. Please choose from {valid_modes}")
# for compatibility with multi-input models, any precipitation dss files or stage hydrograph dss files will be copied to the HECRAS_project_folder

if Operation_Mode == "Build from DSS":
    
    # to do this, we go into each .uXX file in the HECRAS_project_folder and find these lines (IF they exist, they may not be present)
    # 
    # Precipition DSS Filename=
    # DSS Path=
    # DSS File=
    # Met BC=Precipitation|Gridded DSS Filename=
    # This will also copy in any completed results from a previous run
    # Compile these file names into a list and copy them to the HECRAS_project_folder
    # Example: Met BC=Precipitation|Gridded DSS Filename=AORC_May2021_LWIRegion4.dss

    # Create a list of all .uXX files in the HECRAS_project_folder
    HECRAS_project_folder_files = os.listdir(HECRAS_project_folder)
    HECRAS_project_folder_uXX_files = [f for f in HECRAS_project_folder_files if f.endswith(tuple(f".u{str(i).zfill(2)}" for i in range(1, 100)))]
    print("HEC-RAS uXX Files in HECRAS_project_folder:", HECRAS_project_folder_uXX_files)

    # Create a list of all .dss files in the HECRAS_project_folder
    HECRAS_project_folder_dss_files = [f for f in HECRAS_project_folder_files if f.endswith(".dss")]
    print("HEC-RAS DSS Files in HECRAS_project_folder:", HECRAS_project_folder_dss_files)

    for uXX_file in HECRAS_project_folder_uXX_files:
        # Open the .uXX file
        with open(os.path.join(HECRAS_project_folder, uXX_file), 'r') as f:
            # Read all the lines
            lines = f.readlines()
            
            # Create a list of lines that contain the string "Met BC=Precipitation|Gridded DSS Filename="
            met_bc_precipitation_lines = [line for line in lines if "Met BC=Precipitation|Gridded DSS Filename=" in line]
            # Create a list of lines that contain the string "Met BC=Stage Hydrograph|Gridded DSS Filename="
            met_bc_stage_lines = [line for line in lines if "Met BC=Stage Hydrograph|Gridded DSS Filename=" in line]
            
            # Create a list of lines that contain the string "Precipitation DSS Filename="
            precipitation_dss_filename_lines = [line for line in lines if "Precipitation DSS Filename=" in line]
            # Create a list of lines that contain the string "DSS File="
            dss_file_lines = [line for line in lines if "DSS File=" in line]
            
            # Extract the file names from the lines
            met_bc_precipitation_file_names = [line.split("=")[-1].strip() for line in met_bc_precipitation_lines]
            met_bc_stage_file_names = [line.split("=")[-1].strip() for line in met_bc_stage_lines]
            precipitation_dss_file_names = [line.split("=")[-1].strip() for line in precipitation_dss_filename_lines]
            dss_file_file_names = [line.split("=")[-1].strip() for line in dss_file_lines]

            # Combine the lists of file names
            file_names = met_bc_precipitation_file_names + met_bc_stage_file_names + precipitation_dss_file_names + dss_file_file_names

            # Deduplicate the file name list before copying
            unique_file_names = list(set(file_names))

            # Copy the files to the corresponding folder in HECRAS_project_folder
            for file_name in unique_file_names:
                # Construct the full path to the file in the template folder
                template_file_path = os.path.join(HECRAS_template_folder, file_name)
                
                # Check if the file exists in the template folder
                if os.path.exists(template_file_path):
                    # Construct the destination path in the project folder
                    project_file_path = os.path.join(HECRAS_project_folder, file_name)
                    
                    # Check if the destination folder exists, create if missing
                    project_folder_path = os.path.dirname(project_file_path)
                    if not os.path.exists(project_folder_path):
                        os.makedirs(project_folder_path)
                        print(f"Created folder: {project_folder_path}")
                    
                    # Copy the file to the project folder, overwriting existing file
                    shutil.copy2(template_file_path, project_file_path)
                    print(f"Copied {file_name} to {project_file_path}")
                else:
                    print(f"File {file_name} does not exist in {HECRAS_template_folder}. Skipping...")



copied_dss_files = []

if Operation_Mode == "Build from DSS":
    # Create an empty list to store the copied DSS file information
    copied_dss_files = []

    # Iterate over the files in the DSS Import folder
    print(f"Copying DSS files from {DSS_Source_Folder} starting with {DSS_File_Name_Filter_Word} and ending with .dss to {HECRAS_project_folder}")
    for dss_filename in os.listdir(DSS_Source_Folder):
        # If the filename starts with the DSS_File_Name_Filter_Word and ends with .DSS (case insensitive), copy it
        if DSS_File_Name_Filter_Word in dss_filename and DSS_Search_Word in dss_filename and dss_filename.lower().endswith('.dss'):
            try:
                # Define the source and destination paths
                src_path = os.path.join(DSS_Source_Folder, dss_filename)
                dest_path = os.path.join(HECRAS_project_folder, dss_filename)

                # Copy the file from source to destination
                shutil.copy(src_path, dest_path)

                # Extract the run number from the file name
                # Example File Name: May 2021_HMS_Run_1_WF_Final_Cal.dss
                # This run number should be "1", and may be Run_1_ to Run_99_ in the future
                run_number = re.search(r'Run_(\d+)_', dss_filename).group(1) if re.search(r'Run_(\d+)_', dss_filename) else None


                # Append the file information to the list
                copied_dss_files.append({
                    'Full Path': dest_path,
                    'File Name': dss_filename,
                    'Run Number': run_number
                })

                print(f"Copied: {dss_filename}")
            except PermissionError:
                print(f"Permission denied: Unable to copy file {dss_filename}. The file might be in use or you might not have the necessary permissions.")
                print(f"Check if HEC-HMS or HEC-RAS source model is still running?")
    print("Operation completed successfully.")
else:
    print("Operation Mode is set to Run Missing.  Skipping DSS File Copy")

# Convert the list of dictionaries to a DataFrame
copied_dss_files_df = pd.DataFrame(copied_dss_files)
# Print the DataFrame

#print("Copied DSS Files (copied_dss_files_df):)")
#display(copied_dss_files_df)

# Read Existing PRJ and extract .pXX and .uXX file mapping
# Read the .prj file
prj_path = os.path.join(HECRAS_template_folder, HECRAS_prj_file)
with open(prj_path, "r", encoding='latin-1') as prj_file:
    prj_content = prj_file.read()

def extract_plan_and_flow_files(prj_content, output_folder, HECRAS_project_name):
    """Extract .pXX, .uXX, .gXX and their paths from .prj content and output folder."""
    # Extract .pXX references
    pXX_references = re.findall(r"Plan File=(p\d+)", prj_content)
    # Extract .gXX references
    gXX_references = re.findall(r"Geom File=(g\d+)", prj_content)
    
    # For each .pXX, extract the .uXX and .gXX reference and their paths
    data = []
    for pXX in pXX_references:
        # Construct the path based on the provided output folder
        pXX_path = os.path.join(output_folder, f"{HECRAS_project_name}.{pXX}")
        try:
            with open(pXX_path, 'r', encoding='latin-1') as file:
                content = file.read()
                uXX = re.search(r"Flow File=(u\d+)", content).group(1) if re.search(r"Flow File=(u\d+)", content) else None
                gXX = re.search(r"Geom File=(g\d+)", content).group(1) if re.search(r"Geom File=(g\d+)", content) else None
                if uXX and gXX:
                    uXX_path = os.path.join(output_folder, f"{HECRAS_project_name}.{uXX}")
                    gXX_path = os.path.join(output_folder, f"{HECRAS_project_name}.{gXX}")
                    data.append((pXX, uXX, gXX, pXX_path, uXX_path, gXX_path))
        except FileNotFoundError:
            # Skip if the .pXX file is not found
            continue

    return pd.DataFrame(data, columns=['pXX', 'uXX', 'gXX', '.pXX Paths', '.uXX Paths', '.gXX Paths'])

# Call the function and store the result in the specified DataFrame
project_plan_unsteady_mapping_df = extract_plan_and_flow_files(prj_content, HECRAS_template_folder, HECRAS_project_name)

project_plan_unsteady_mapping_df['.gXX.hdf Paths'] = project_plan_unsteady_mapping_df['.gXX Paths'] + '.hdf'


# Show the DataFrame
print("Existing plans and geometries dataframe: project_plan_unsteady_mapping_df")
display(project_plan_unsteady_mapping_df)

In [ ]:
#7 If Build from DSS, Create Proposed Plan and Unsteady File Numbers DataFrame

def extract_numbers_from_prj(prj_content):
    """
    Extract all unsteady and plan file numbers from the given PRJ content.
    Returns two lists containing unsteady and plan file numbers, respectively.
    """
    unsteady_pattern = re.compile(r"Unsteady File=u(\d+)")
    plan_pattern = re.compile(r"Plan File=p(\d+)")
    geometry_pattern = re.compile(r"Geom File=g(\d+)")
    
    unsteady_matches = unsteady_pattern.findall(prj_content)
    plan_matches = plan_pattern.findall(prj_content)
    geometry_matches = geometry_pattern.findall(prj_content)
    print("unsteady_matches: ", unsteady_matches) #DEBUG ONLY
    
    unsteady_numbers = [int(num) for num in unsteady_matches]
    print("unsteady_numbers: ", unsteady_numbers) #DEBUG ONLY
    plan_numbers = [int(num) for num in plan_matches]
    print("plan_numbers: ", plan_numbers) #DEBUG ONLY
    geometry_numbers = [int(num) for num in geometry_matches]
    print("geometry_numbers: ", geometry_numbers) #DEBUG ONLY
    
    return unsteady_numbers, plan_numbers, geometry_numbers

def display_existing_plan_unsteady(prj_content):
    """
    Creates and displays a dataframe with existing unsteady and plan file numbers based on the given PRJ content.
    """
    # Extract existing unsteady and plan numbers from the .prj content
    existing_unsteady_numbers, existing_plan_numbers, existing_geometry_numbers = extract_numbers_from_prj(prj_content)
    print("existing_unsteady_numbers: ", existing_unsteady_numbers) #DEBUG ONLY
    print("existing_plan_numbers: ", existing_plan_numbers) #DEBUG ONLY
    print("existing_geometry_numbers: ", existing_geometry_numbers) #DEBUG ONLY


    # Convert to extensions
    existing_unsteady_extensions = [f"u{num:02}" for num in existing_unsteady_numbers]
    print("existing_unsteady_extensions: ", existing_unsteady_extensions) #DEBUG ONLY
    existing_plan_extensions = [f"p{num:02}" for num in existing_plan_numbers]
    print("existing_plan_extensions: ", existing_plan_extensions) #DEBUG ONLY
    existing_geometry_extensions = [f"g{num:02}" for num in existing_geometry_numbers]
    print("existing_geometry_extensions: ", existing_geometry_extensions) #DEBUG ONLY

    # Construct and display the DataFrame
    existing_data = [("Unsteady File", ext) for ext in existing_unsteady_extensions] + [("Plan File", ext) for ext in existing_plan_extensions] + [("Geometry File", ext) for ext in existing_geometry_extensions]
    existing_df = pd.DataFrame(existing_data, columns=['Type', 'File Extension'])
    
    #print("display existing_df inside loop for debugging") # DEBUG ONLY 
    #display(existing_df) # DEBUG ONLY 

    # Return the DataFrame instead of displaying it
    return existing_df

def get_next_unique_number(existing_numbers, start_number=1, max_number=99):
    """
    Returns the next unique number based on the existing numbers, starting from start_number up to max_number.
    """
    current_number = start_number
    while current_number <= max_number:
        if current_number not in existing_numbers:
            return current_number
        current_number += 1
    return None  # Return None if all numbers up to max_number are taken


def create_new_plan_data(prj_content, CopiedDssFilesDataFrame):
    """
    Creates a dataframe with new unique plan and unsteady file numbers based on the given PRJ content and DSS files DataFrame.
    Provides a warning if the total number of plans exceeds 99.
    """
    existing_unsteady_numbers, existing_plan_numbers, existing_geometry_numbers = extract_numbers_from_prj(prj_content)
    
    # Check if total plans exceed 99
    total_plans = len(existing_plan_numbers) + len(CopiedDssFilesDataFrame)
    print(f"Total Plans: {total_plans}")
    if total_plans > 99:
        raise Warning("HEC-RAS only allows 99 plans per project folder. Reduce the number of DSS files or delete existing plan files.")

    NewPlanData = []
    for _, row in CopiedDssFilesDataFrame.iterrows():
        file_name = row['File Name']
        new_plan_title = file_name.replace(DSS_Search_Word, DSS_Replace_Word).replace(".dss", "").replace(".DSS", "")

        unsteady_number = get_next_unique_number(existing_unsteady_numbers)
        plan_number_cnpd = get_next_unique_number(existing_plan_numbers)
        geometry_number = get_next_unique_number(existing_geometry_numbers)
        
        # print("(inside loop) geometry_number: ", geometry_number) # DEBUG ONLY
        
        if unsteady_number is None or plan_number_cnpd is None:
            raise ValueError("Exceeded the maximum number of unique unsteady or plan files (99).")

        unsteady_extension = f"u{unsteady_number:02}"
        plan_extension = f"p{plan_number_cnpd:02}"
        geometry_extension = f"g{geometry_number:02}"
        
        if Enable_Infiltration_Overrides:
            NewPlanData.append((file_name, new_plan_title, unsteady_extension, plan_extension, geometry_extension))
        else:
            NewPlanData.append((file_name, new_plan_title, unsteady_extension, plan_extension))

        existing_unsteady_numbers.append(unsteady_number)
        existing_plan_numbers.append(plan_number_cnpd)
        existing_geometry_numbers.append(geometry_number)  # This line is crucial
        
    # Only return new geometry numbers if Enable_Infiltration_Overrides == True
    if Enable_Infiltration_Overrides:
        print("Enable_Infiltration_Overrides = True, so adding geometry numbers")
        existing_geometry_numbers.append(geometry_number)
        NewPlanDataFrame = pd.DataFrame(NewPlanData, columns=['DSS File', 'New Plan Title', 'Unsteady File Extension', 'Plan File Extension', 'Geometry File Extension'])
    else:
        NewPlanDataFrame = pd.DataFrame(NewPlanData, columns=['DSS File', 'New Plan Title', 'Unsteady File Extension', 'Plan File Extension'])
    return NewPlanDataFrame



if Operation_Mode == "Build from DSS":
    # Build PRJ file path (assuming HECRAS_project_folder and HECRAS_prj_file are defined)
    prj_path = os.path.join(HECRAS_project_folder, HECRAS_prj_file)
    print("PRJ File Path:", prj_path)

    print ("Reading PRJ file to extract existing Plan File and Unsteady File Numbers")
    with open(prj_path, "r") as prj_file:
        prj_content = prj_file.read()

    # Display the existing plan and unsteady file numbers
    print("")
    print("Existing Plan and Unsteady File Numbers (prj_content dataframe):")
    display_existing_plan_unsteady(prj_content)
    print("prj_content dataframe:")
    #display(prj_content) # DEBUG ONLY


    # Create and display the new plan dataframe
    print("")
    print("New Plan and Unsteady File Numbers:")
    new_plan_title_df = create_new_plan_data(prj_content, copied_dss_files_df)
 

    # Modify the new_plan_title_df to add required columns
    new_plan_title_df["Short Identifier"] = new_plan_title_df["New Plan Title"]
    new_plan_title_df["Flow File"] = "u" + new_plan_title_df["Unsteady File Extension"].str[1:]
    new_plan_title_df["Plan File Name"] = HECRAS_project_name + "." + new_plan_title_df["Plan File Extension"]
    new_plan_title_df["Unsteady File Name"] = HECRAS_project_name + "." + new_plan_title_df["Unsteady File Extension"]
    
    # Use .apply with lambda for constructing file paths
    new_plan_title_df["Plan File Path"] = new_plan_title_df["Plan File Extension"].apply(lambda x: os.path.join(HECRAS_project_folder, HECRAS_project_name + "." + x))
    new_plan_title_df["Unsteady File Path"] = new_plan_title_df["Unsteady File Extension"].apply(lambda x: os.path.join(HECRAS_project_folder, HECRAS_project_name + "." + x))
    if Enable_Infiltration_Overrides == True:
        new_plan_title_df["Geometry File Path"] = new_plan_title_df["Geometry File Extension"].apply(lambda x: os.path.join(HECRAS_project_folder, HECRAS_project_name + "." + x))
    new_plan_title_df["HMS Input DSS File"] = new_plan_title_df["DSS File"]
    new_plan_title_df["RAS Output DSS File"] = new_plan_title_df["DSS File"].str.replace(DSS_Search_Word, DSS_Replace_Word, regex=False)
    print("New Plan and Unsteady Dataframe:")
    print("new_plan_title_df:")
    display(new_plan_title_df)

    # If new_plan_title_df is empty, raise an exception "Check your paths, search terms and files.  No plans generated from DSS files."
    if new_plan_title_df.empty:
        raise Exception("Check your paths, search terms and files.  No plans generated from DSS files.")

    # Write the DataFrame to the output CSV file
    output_csv_path = os.path.join(HECRAS_project_folder, plan_title_csv_file_name)
    new_plan_title_df.to_csv(output_csv_path, index=False)
    print(f"New Plan and Unsteady Dataframe was written to CSV for user inspection: {output_csv_path}")
   

    
if Operation_Mode == "Run Missing":

    # NO FURTHER CHANGES WILL BE MADE TO THE OUTPUT FOLDER
    # ONLY SCANNING FOR DSS AND BUILDING A LIST OF RUNS THAT NEED TO BE EXECUTED
       

    # Create an empty list to store the DSS file information
    RAS_exist_output_dss_files = []

    # Iterate and find DSS files in HEC RAS Output folder
    print(f"Scanning {HECRAS_project_folder} for DSS files containing RAS in the name")
    for file_name in os.listdir(HECRAS_project_folder):
        # If the filename contains "RAS" and ends with .DSS (case insensitive), copy it
        if file_name.lower().endswith('.dss'):
            try:
                # Define the source and destination paths
                src_path = os.path.join(HECRAS_project_folder, file_name)

                # Extract the run number from the file name
                # Example File Name: May 2021_HMS_Run_1_WF_Final_Cal.dss
                # This run number should be "1", and may be Run_1_ to Run_99_ in the future
                run_number = re.search(r'Run_(\d+)_', file_name).group(1) if re.search(r'Run_(\d+)_', file_name) else None


                # Append the file information to the list
                RAS_exist_output_dss_files.append({
                    'Full Path': src_path,
                    'File Name': file_name,
                    'Run Number': run_number
                })

                print(f"Found: {file_name}")
            except PermissionError:
                print(f"Permission denied: Unable to copy file {file_name}. The file might be in use or you might not have the necessary permissions.")
                print(f"Check if HEC-HMS is running")

    # Convert the list of dictionaries to a DataFrame
    RAS_exist_output_dss_files_df = pd.DataFrame(RAS_exist_output_dss_files)
    # Print the DataFrame
    print("RAS_exist_output_dss_files_df:")
    display(RAS_exist_output_dss_files_df)

    # In Run Missing Mode, instead of building new plan numbers in previous steps, we are simply reading the prj, .pxx and .uxx files inside of the HECRAS_project_folder  
    prj_path = os.path.join(HECRAS_project_folder, HECRAS_prj_file)


    print("PRJ File Path:", prj_path)

    print ("Reading PRJ file to extract existing Plan File and Unsteady File Numbers")
    with open(prj_path, "r") as prj_file:
        prj_content = prj_file.read()

    # Display the existing plan and unsteady file numbers
    print("")
    existing_df = display_existing_plan_unsteady(prj_content)

    # print("display existing_df outside loop for debugging")
    # print(existing_df)

    # Now we need to peek in each unsteady file in the RAS output folder and read the DSS file name
    # The DSS file is contained on a line starting with "DSS File=", for example "DSS File=May 2021_RAS_Run_12_WF_Final_Cal.dss"
    # From the DSS file name, we can determine the Run Number
    # Using each Plan File entry in prj_content, we can determine the RAS Output DSS File Name
    # These entries should be compiled in the dataframe Run_Missing_Runlist_df

    # For each row under "File" matching "Plan File" in prj_content, we can determine the RAS Output DSS File Name

    # plan file name = os join (HECRAS_project_folder, HECRAS_project_name + "." + Extension from prj_content)
    run_missing_runlist_accumulator = []
    for index, row in existing_df.iterrows():
        if row['Type'] == 'Plan File':
            plan_file_name = os.path.join(HECRAS_project_folder, HECRAS_project_name + "." + row['File Extension'])
            dss_file_found = False
            try:
                with open(plan_file_name, "r", encoding='ISO-8859-1') as plan_file:
                    for line in plan_file:
                        if line.startswith("DSS File="):
                            dss_file_path = os.path.join(HECRAS_project_folder, line.strip().split('=')[1])
                            if os.path.isfile(dss_file_path):
                                dss_file_found = True
                            break
            except UnicodeDecodeError as e:
                print(f"Error reading {plan_file_name}: {e} try editing the script to use a different encoding")
            if not dss_file_found:
                run_missing_runlist_accumulator.append({'Plan File Extension': row['File Extension']})
               

    # Initialize the Run_Missing_Runlist_df dataframe
    Run_Missing_Runlist_df = pd.DataFrame(columns=['Plan File Extension'])

    Run_Missing_Runlist_df = pd.concat([Run_Missing_Runlist_df, pd.DataFrame(run_missing_runlist_accumulator)])
    print("Run_Missing_Runlist_df:")
    display(Run_Missing_Runlist_df)
    # In this dataframe, there are double entries for each plan file extension, and we need to determine how this happened and how to fix it




In [ ]:
#8 If "Build from DSS", update the plan (.pxx), unsteady (.uxx) and prj files with new Short Identifiers, Plan Titles, DSS Files, Flow Files, Etc.

if Operation_Mode == "Build from DSS":
    #8 Select ".p01" or first existing plan as template and copy for each new plan, and Update the plan (.pxx) files with new Short Identifiers, Plan Titles, DSS Files, and Flow Files
    # Update PRJ File with new Unsteady and Plan File entries
    # Update Unsteady (.uXX) files with "DSS File" and "Flow Title" entries
    
    
    # default plan should be based on Plan_Number, and provide a warning if ".pXX" is not found
    default_plan = "p" + Plan_Number
    print("Default Plan:", default_plan)
    # provide a warning if ".pXX" is not found
    if default_plan not in project_plan_unsteady_mapping_df.values:
        print(f"Warning: {default_plan} not found in {HECRAS_project_folder}.  Please check that your default plan number exists")
        print("")
    
    #print("\nNote: Overriding the Source Plan requires re-executing all following cells.")
    
    def copy_source_files_to_dest(source_plan_file, source_unsteady_file, dataframe):
        """Copy the source plan and unsteady files to the destinations based on the provided dataframe."""
    
        # Iterate over the dataframe to get the proposed .pXX and .uXX file names
        for _, row in dataframe.iterrows():
            dest_plan_file = os.path.join(HECRAS_project_folder, row["Plan File Name"])
            dest_unsteady_file = os.path.join(HECRAS_project_folder, row["Unsteady File Name"])
    
            # Copy the source files to the proposed destinations
            shutil.copy(source_plan_file, dest_plan_file)
            print(f"Copied {source_plan_file} to {dest_plan_file}")
            shutil.copy(source_unsteady_file, dest_unsteady_file)
            print(f"Copied {source_unsteady_file} to {dest_unsteady_file}") 
            print("")
    
    # Change source_plan_file to populate from prj_content dataframe
    '''Existing Plan and Unsteady File Numbers (prj_content dataframe):
    Type	File Extension
    0	Unsteady File	u01
    1	Unsteady File	u02
    2	Plan File	p01
    3	Plan File	p02
    '''
    
    ''' project_plan_unsteady_mapping_df
        pXX	uXX	.pXX Paths	.uXX Paths
    0	p01	u01	H:\2202134.00C_LWI_Region4\05C_WestForkCalcasi...	H:\2202134.00C_LWI_Region4\05C_WestForkCalcasi...
    1	p02	u02	H:\2202134.00C_LWI_Region4\05C_WestForkCalcasi...	H:\2202134.00C_LWI_Region4\05C_WestForkCalcasi...
    '''


    # Attempt to get the source plan file path
    source_plan_file_path = project_plan_unsteady_mapping_df.loc[project_plan_unsteady_mapping_df['pXX'] == default_plan, '.pXX Paths']
    if not source_plan_file_path.empty:
        source_plan_file = source_plan_file_path.values[0]
    else:
        print(f"Warning: {default_plan} plan file not found in project_plan_unsteady_mapping_df.")
        # Handle the missing plan file case here
    
    display(project_plan_unsteady_mapping_df)

    '''example project_plan_unsteady_mapping_df
    
    pXX	uXX	.pXX Paths	.uXX Paths
    0	p02	u01	c:\Temp\RAS1D_May2021-Executed-wide\WF_WestFor...	c:\Temp\RAS1D_May2021-Executed-wide\WF_WestFor...
    
    '''
    print("Default Plan:", default_plan)


    matching_row = project_plan_unsteady_mapping_df.loc[project_plan_unsteady_mapping_df['pXX'] == default_plan]
    print("matching_row: ", matching_row)

    if not matching_row.empty:
        source_unsteady_file = matching_row['.uXX Paths'].values[0]
        print("source_unsteady_file", source_unsteady_file)
        source_plan_file = matching_row['.pXX Paths'].values[0]
        print("source_plan_file: ", source_plan_file)
    else:
        print(f"Warning: Corresponding unsteady file for {default_plan} not found in project_plan_unsteady_mapping_df.")
        # Handle the missing unsteady file case here

    if 'source_plan_file' in locals() or 'source_plan_file' in globals():
        print("Source Plan File:", source_plan_file)
    else:
        raise Exception("Check User Inputs, the source plan number is incorrect")
    print("Source Unsteady File:", source_unsteady_file)
    
    # Call the copying function
    copy_source_files_to_dest(source_plan_file, source_unsteady_file, new_plan_title_df)
    
    # Update the plan (.pxx) files with new Short Identifiers, Plan Titles, DSS Files, and Flow Files

    # Additional logic for Enable_Infiltration_Overrides == True
    # Edit the existing "Geom File=" line(Example: "Geom File=g01") with the new "Geom File=" line (Example: "Geom File=g02")
    # The line should be populated from the new_plan_title_df dataframe using column "Geometry File Extension"


    for _, row in new_plan_title_df.iterrows():
        plan_file_path = os.path.join(HECRAS_project_folder, row["Plan File Name"])
    
        if os.path.exists(plan_file_path):
            with open(plan_file_path, "r", encoding="ISO-8859-1") as plan_file:
                content = plan_file.read()
            print(f"Updating {plan_file_path}")
            content = re.sub(r"Plan Title=.*", f"Plan Title={row['New Plan Title']}", content)
            print(f"Updating Plan Title to {row['New Plan Title']}")
            content = re.sub(r"Short Identifier=.*", f"Short Identifier={row['Short Identifier']}", content)
            print(f"Updating Short Identifier to {row['Short Identifier']}")
            content = re.sub(r"DSS File=.*", f"DSS File={os.path.basename(row['RAS Output DSS File'])}", content)
            print(f"Updating DSS File to {os.path.basename(row['RAS Output DSS File'])}")
            content = re.sub(r"Flow File=.*", f"Flow File={row['Flow File']}", content)
            print(f"Updating Flow File to {row['Flow File']}")
            # Only update the Geom File if Enable_Infiltration_Overrides == True
            if Enable_Infiltration_Overrides:
                content = re.sub(r"Geom File=.*", f"Geom File={row['Geometry File Extension']}", content)
                print(f"Updating Geom File to {row['Geometry File Extension']}")

            with open(plan_file_path, "w") as plan_file:
                plan_file.write(content)
                print(f"Successfully updated {plan_file_path}")
                print("")
        else:
            print(f"Plan file not found: {plan_file_path}")
    
    
    # Update PRJ File with new Unsteady and Plan File entries
    
    # Read the .prj file content
    prj_file_path = os.path.join(HECRAS_project_folder, HECRAS_prj_file)
    with open(prj_file_path, 'r') as prj_file:
        prj_content = prj_file.read()
    
    # Use Dataframe to create New Unsteady File and Plan File entries
    unsteady_entries = [f"Unsteady File={row['Unsteady File Extension']}" for _, row in new_plan_title_df.iterrows()]
    print("Unsteady Entries (unsteady_entries dataframe):")
    
    
    #display(unsteady_entries) # DEBUG ONLY
    plan_entries = [f"Plan File={row['Plan File Extension']}" for _, row in new_plan_title_df.iterrows()]
    #print("Plan Entries (plan_entries dataframe):") # DEBUG ONLY
    #display(plan_entries) # DEBUG ONLY
    
    # Insert Unsteady File and Plan File entries after the existing entries in the PRJ file content
    unsteady_pattern = r"(Unsteady File=.*\n)"
    plan_pattern = r"(Plan File=.*\n)"
    prj_content = re.sub(unsteady_pattern, r"\1" + '\n'.join(unsteady_entries) + '\n', prj_content)
    prj_content = re.sub(plan_pattern, r"\1" + '\n'.join(plan_entries) + '\n', prj_content)

    # remove any duplicate entries
    prj_content = re.sub(r"Unsteady File=(u\d+).*\n\1", r"Unsteady File=\1", prj_content)
    prj_content = re.sub(r"Plan File=(p\d+).*\n\1", r"Plan File=\1", prj_content)




    ''' Example of PRJ Content (original)
    Proj Title=WF_WestForkCalcasieu
    Current Plan=p02
    Default Exp/Contr=0.3,0.1
    English Units
    Geom File=g01
    Unsteady File=u01
    Unsteady File=u02
    Plan File=p01
    Plan File=p02
    Y Axis Title=Elevation
    X Axis Title(PF)=Main Channel Distance
    X Axis Title(XS)=Station


    '''
    ''' Example of PRJ Content (after insertions)
    Proj Title=WF_WestForkCalcasieu
    Current Plan=p02
    Default Exp/Contr=0.3,0.1
    English Units
    Geom File=g01
    Geom File=g02
    Geom File=g03
    Geom File=g04
    Geom File=g05
    Geom File=g06
    Geom File=g07
    Unsteady File=u01
    Unsteady File=u02
    Unsteady File=u03
    Unsteady File=u04
    Unsteady File=u05
    Unsteady File=u06
    Unsteady File=u07
    Unsteady File=u08
    Plan File=p01
    Plan File=p02
    Plan File=p03
    Plan File=p04
    Plan File=p05
    Plan File=p06
    Plan File=p07
    Plan File=p08
    Y Axis Title=Elevation
    X Axis Title(PF)=Main Channel Distance
    X Axis Title(XS)=Station


    '''






    
    # Write the updated content to the PRJ file
    with open(prj_file_path, 'w') as prj_file:
        prj_file.write(prj_content)
        print(f"Successfully updated {prj_file_path}")
    
    
    # Generate the HEC-Cmdr_New_Plan_Mapping.html file
    html_output_path = os.path.join(HECRAS_project_folder, "HEC-Cmdr_New_Plan_Mapping.html")
    with open(html_output_path, "w") as html_file:
        
        html_file.write("<h2>Existing Project Plan and Unsteady File Mapping:</h1>")
        project_plan_unsteady_mapping_df.to_html(buf=html_file, index=False)
    
    
        html_file.write("<h1>New Plan Mapping</h1>")
        new_plan_title_df.to_html(buf=html_file, index=False) 
        print(f"Successfully generated {html_output_path}")
        
    
    # Update Unsteady (.uXX) files with "DSS File" and "Flow Title" entries
       
    # Update the .uXX files with the new values
    for _, row in new_plan_title_df.iterrows():
        # Unpack row based on the new structure
        hms_input_dss_file = row['HMS Input DSS File']
        new_plan_title = row['New Plan Title']
        unsteady_file_extension = row['Unsteady File Extension']
        unsteady_file_name = row['Unsteady File Name']

    
        # Check if the corresponding .uXX file exists in the specified folder
        unsteady_file_path = os.path.join(HECRAS_project_folder, unsteady_file_name)
        if os.path.exists(unsteady_file_path):
            # Read the file content
            with open(unsteady_file_path, "r") as unsteady_file:
                unsteady_file_content = unsteady_file.read()
    
            # Update the values for "DSS File" and "Flow Title"
            # Using HMS Input DSS File instead of DSS File
            unsteady_file_content = re.sub(r"DSS File\s*=\s*\S+", f"DSS File={hms_input_dss_file}", unsteady_file_content)
            print(f"Updating DSS File to {hms_input_dss_file}")
            unsteady_file_content = re.sub(r"Flow Title\s*=\s*\S+", f"Flow Title={new_plan_title}", unsteady_file_content)
            print(f"Updating Flow Title to {new_plan_title}")

            # for each DSS Path= entry, they follow the same pattern: DSS Path=/*/*/*/*/*/*/ (Example: DSS Path=/BECKWCR 4.1/24/FLOW//1Hour/M*/ ) and the last entry should be deleted and replaced
            # For example, the entry above should be edited to DSS Path=/BeckwCr 4.1/24/FLOW/01May2021/1Hour/May 2021_RAS1D_Run_4_WF_Final_Cal/
            # The last entry should be the HMS Input DSS File Name without the .dss extension 

            # Find all DSS Path= entries
            dss_path_entries = re.findall(r"DSS Path=.*", unsteady_file_content)
            print("dss_path_entries: ", dss_path_entries)
            # Replace the last entry (after the 6th "/" and before the last (seventh) "/") with the HMS Input DSS File Name without the .dss extension
            if Override_DSS_Path:
                for dss_path_entry in dss_path_entries:
                    # Split the entry by "/"
                    dss_path_entry_split = dss_path_entry.split("/")
                    # Check if the split entry has the expected number of parts
                    if len(dss_path_entry_split) >= 7:
                        # Replace the last entry with the HMS Input DSS File Name without the .dss extension
                        dss_path_entry_split[-2] = hms_input_dss_file.replace(".dss", "")
                        # Join the split entry back together
                        dss_path_entry = "/".join(dss_path_entry_split)
                        # Replace the entry in the unsteady file content
                        unsteady_file_content = re.sub(r"DSS Path=.*", dss_path_entry, unsteady_file_content)
                        print(f"Updating DSS Path to {dss_path_entry}")
                    else:
                        print(f"Unexpected DSS Path format: {dss_path_entry}")
  
            # Write the updated content back to the .uXX file
            with open(unsteady_file_path, "w") as unsteady_file:
                unsteady_file.write(unsteady_file_content)
                print(f"Successfully updated {unsteady_file_path}")
        else:
            print(f"Unable to find Unsteady file: {unsteady_file_path}")


# IF "Run Missing", NOTHING SHOULD BE DONE HERE.  Just deploy and batch execute.  


In [ ]:
#9 If Enable_Infiltration_Overrides = True, Copy Geometries and Update Geometry HDF Files with new Infiltration Grid Base Overrides

def scale_infiltration_data(hdf_file_path, infiltration_df, scale_md, scale_id, scale_pr):
    # Define the HDF path to be overwritten
    hdf_path_to_overwrite = '/Geometry/Infiltration/Base Overrides'
    print("scale_pr: ", scale_pr)
    # Scale the data as required
    infiltration_df['Maximum Deficit'] *= scale_md
    infiltration_df['Initial Deficit'] *= scale_id
    infiltration_df['Potential Percolation Rate'] *= scale_pr
    
    

    # Open the HDF file
    with h5py.File(hdf_file_path, 'a') as hdf_file:
        # Check if the dataset exists and delete it if it does
        if hdf_path_to_overwrite in hdf_file:
            del hdf_file[hdf_path_to_overwrite]
        
        # Create a new dataset for infiltration data
        dt = np.dtype([('Land Cover Name', 'S7'), 
                    ('Maximum Deficit', 'f4'), 
                    ('Initial Deficit', 'f4'), 
                    ('Potential Percolation Rate', 'f4')])
        
        # Convert DataFrame to structured array
        structured_array = np.zeros(infiltration_df.shape[0], dtype=dt)
        structured_array['Land Cover Name'] = np.array(infiltration_df['Name'].astype(str).values.astype('|S7'))
        structured_array['Maximum Deficit'] = infiltration_df['Maximum Deficit'].values.astype(np.float32)
        structured_array['Initial Deficit'] = infiltration_df['Initial Deficit'].values.astype(np.float32)
        structured_array['Potential Percolation Rate'] = infiltration_df['Potential Percolation Rate'].values.astype(np.float32)
        
        # Define maxshape as None for unlimited dimensions
        maxshape = (None,)
        
        # Create the dataset with the specified dtype, and compression options
        hdf_file.create_dataset(
            hdf_path_to_overwrite, 
            data=structured_array, 
            dtype=dt, 
            compression='gzip', 
            compression_opts=1, 
            chunks=(100,),  # Chunk size
            maxshape=maxshape  # Allow unlimited max shape
        )
        
        # Return the scaled DataFrame
        return infiltration_df

# IF Enable_Infiltration_Overrides = True AND Operation_Mode = "Build from DSS", THEN
if Enable_Infiltration_Overrides and Operation_Mode == "Build from DSS":   

    # Read run parameters from CSV file
    try:
        user_calibration_df = pd.read_csv(user_calibration_runs_csv_fullpath, dtype={'user_run_number_from_csv': int})
    except Exception as e:
        raise Exception("Check user_calibration_runs_csv_fullpath in User Inputs Section.  Ensure CSV follows example") from e
    def load_user_calibration_csv_data(file_path):
        user_calibration_df = pd.read_csv(file_path, dtype={'user_run_number_from_csv': int})  # Directly reading csv using pandas to return DataFrame
        return user_calibration_df # Show DataFrame

    load_user_calibration_csv_data(user_calibration_runs_csv_fullpath)
    print("User Calibration Data from CSV with parameter scaling values (user_calibration_df):")
    display(user_calibration_df)

    # Read Infiltration Grid CSV
    Infiltration_From_RAS2D_df = pd.read_csv(Infiltration_From_RASMapper_csv)
    print("\nInfiltration Data from CSV (This should be unscaled data values) as Infiltration_From_RAS2D_df:")
    display(Infiltration_From_RAS2D_df)


    # add prefix to default plan number
    plan_number_with_prefix = 'p' + Plan_Number  # Add 'p' prefix to Plan_Number
    # Define the source file paths
    source_geometry_file = project_plan_unsteady_mapping_df.loc[project_plan_unsteady_mapping_df['pXX'] == plan_number_with_prefix, '.gXX Paths'].iloc[0]
    soure_geometry_hdf_file = project_plan_unsteady_mapping_df.loc[project_plan_unsteady_mapping_df['pXX'] == plan_number_with_prefix, '.gXX.hdf Paths'].iloc[0]

    # Loop through each row in new_plan_title_df and copy files
    for _, row in new_plan_title_df.iterrows():
        # Destination file paths based on the current row in new_plan_title_df      
        dest_geometry_file = os.path.join(HECRAS_project_folder, HECRAS_project_name + "." + row["Geometry File Extension"])
        dest_geometry_hdf_file = os.path.join(HECRAS_project_folder, HECRAS_project_name + "." + row["Geometry File Extension"] + ".hdf")

        shutil.copy(source_geometry_file, dest_geometry_file)
        print(f"Copied {os.path.basename(source_geometry_file)} to {os.path.basename(dest_geometry_file)}")

        shutil.copy(soure_geometry_hdf_file, dest_geometry_hdf_file)
        print(f"Copied {os.path.basename(soure_geometry_hdf_file)} to {os.path.basename(dest_geometry_hdf_file)}")
        print("")



if Enable_Infiltration_Overrides and Operation_Mode == "Build from DSS":

    # add a new column to new_plan_title_df with the Run Number (extract from New Plan Title, format _Run_X_ or _Run_XX_)
    # if the pattern can't be found, raise and exception and notify the user "Unable to extract Run Numbers from New Plan Title.  To use infiltration overrides, please use the following naming convention:  <Project Name>_Run_XX_<Short Identifier>"
    # if the pattern is found, extract the Run Number and add it to the new column
    try:
        # Extracting run numbers from 'New Plan Title'
        new_plan_title_df['Run_Number'] = new_plan_title_df['New Plan Title'].str.extract(r'Run_(\d{1,2})_')

        # Replace NaN values with a default value to indicate missing or unextractable data
        # Here, -1 is used as an example. You can choose a different value if preferred.
        new_plan_title_df['Run_Number'] = new_plan_title_df['Run_Number'].fillna(-1).astype(int)
    except ValueError:
        raise Exception("Unable to extract Run Numbers from New Plan Title.  To use infiltration overrides, please use the following naming convention:  <Project Name>_Run_XX_<Short Identifier>")

    # Display the updated new_plan_title_df with the Run Number (debug only)
    # display(new_plan_title_df)

    # now, the Run Number can be matched with user_run_number_from_csv in user_calibration_df
    # for each row in new_plan_title_df, match the Run Number with user_run_number_from_csv and add maximum_deficit_scale, initial_deficit_scale and percolation_rate_scale to new_plan_title_df
    # All should have run numbers or an exception should have been raised above

    for index, row in new_plan_title_df.iterrows():
        run_number = row['Run_Number']
        matching_row = user_calibration_df.loc[user_calibration_df['user_run_number_from_csv'] == run_number]

        # Check if matching_row is empty
        if not matching_row.empty:
            new_plan_title_df.loc[index, 'maximum_deficit_scale'] = matching_row['maximum_deficit_scale'].values[0]
            new_plan_title_df.loc[index, 'initial_deficit_scale'] = matching_row['initial_deficit_scale'].values[0]
            new_plan_title_df.loc[index, 'percolation_rate_scale'] = matching_row['percolation_rate_scale'].values[0]
        else:
            # Display the problematic row for user inspection
            print("Problematic row in new_plan_title_df:")
            print(row)

            # Raise an exception with a descriptive message
            raise ValueError(f"No matching calibration data found for run number {run_number}. Please check that the DSS file names and CSV Run Numbers match.")

    # display the updated new_plan_title_df with calibration data
    display(new_plan_title_df)

    for index, row in new_plan_title_df.iterrows():
        # Step 1: Get file names and paths
        geometry_file_extension = row['Geometry File Extension']
        hdf_file_name = f"{HECRAS_project_name}.{geometry_file_extension}.hdf"
        hdf_file_path = os.path.join(HECRAS_project_folder, hdf_file_name)
        ascii_file_name = f"{HECRAS_project_name}.{geometry_file_extension}"
        ascii_file_path = os.path.join(HECRAS_project_folder, ascii_file_name)

        # Step 2: Scale the infiltration data
        scale_md = row['maximum_deficit_scale']
        scale_id = row['initial_deficit_scale']
        scale_pr = row['percolation_rate_scale']
        print(f"Scaling infiltration data for {hdf_file_name} with the following factors: Maximum Deficit: {scale_md}, Initial Deficit: {scale_id}, Potential Percolation Rate: {scale_pr}")
        scaled_df = scale_infiltration_data(hdf_file_path, Infiltration_From_RAS2D_df.copy(), scale_md, scale_id, scale_pr)

        # Optionally display the scaled DataFrame
        print(f"Scaled infiltration data for {hdf_file_name}:")
        
        # Export to csv
        scaled_df.to_csv(os.path.join(HECRAS_project_folder, f"{hdf_file_name}.csv"), index=False)

        display(scaled_df.head(100))

        # Step 3: Update geometry title in the ASCII file
        new_plan_title = row['New Plan Title']
        with open(ascii_file_path, 'r') as file:
            lines = file.readlines()

        with open(ascii_file_path, 'w') as file:
            for line in lines:
                if line.startswith('Geom Title='):
                    file.write(f'Geom Title={new_plan_title}\n')
                else:
                    file.write(line)

        print(f"Updated geometry title in {ascii_file_name} to {new_plan_title}")



    # Assuming HECRAS_project_folder and HECRAS_prj_file are defined
    prj_file_path = os.path.join(HECRAS_project_folder, HECRAS_prj_file)

    # Read the .prj file content into a list of lines
    with open(prj_file_path, 'r') as prj_file:
        lines = prj_file.readlines()

    # Find all .gxx files in the project folder
    gxx_files = [file for file in os.listdir(HECRAS_project_folder) if file.endswith(tuple(f'.g{str(i).zfill(2)}' for i in range(1, 100)))]
    gxx_extensions = [file.split('.')[1] for file in gxx_files]

    # Create a list of existing geometry numbers (with formatting)
    existing_geometry_extensions = [re.search(r"g\d+", line).group() for line in lines if "Geom File=" in line]

    # Find the missing geometry extensions
    missing_geometry_extensions = list(set(gxx_extensions) - set(existing_geometry_extensions))

    # Sort the missing geometry extensions in ascending numerical order
    missing_geometry_extensions.sort(key=lambda x: int(re.search(r'(\d+)', x).group()))

    # Find the index of the last "Geom File=" line before the new insertions
    last_existing_geom_index = max(i for i, line in enumerate(lines) if "Geom File=" in line and re.search(r"g\d+", line).group() in existing_geometry_extensions)

    # Insert the new "Geom File=" lines in ascending order after the last existing geometry file
    for geom in missing_geometry_extensions:
        last_existing_geom_index += 1
        lines.insert(last_existing_geom_index, f"Geom File={geom}\n")

        

    # Write the updated content back to the PRJ file
    with open(prj_file_path, 'w') as prj_file:
        prj_file.writelines(lines)
        print(f"Successfully updated {prj_file_path} with missing geometry files")






In [ ]:
#10 Delete Existing Files and Directories in HECRAS_Deploy_Targets and Copy Folders to HECRAS_Deploy_Targets (with filtering)

def clear_directory(dir_path):
    """Clears the directory given the path."""
    
    if not os.listdir(dir_path):
        print(f"No Files in the Target Folder ({dir_path}), No Folder Deletion Required")
        return

    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        
        if not filename.startswith(Folder_Safety_Prefix):
            if os.path.isfile(file_path):
                print(f'Skipping file: {file_path} - Not matching the safety prefix.')
            elif os.path.isdir(file_path):
                print(f'Skipping directory: {file_path} - Not matching the safety prefix.')
            continue
        
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
            print(f'Successfully deleted: {file_path}')
        except Exception as e:
            raise Exception("Cannot clear remote deploy targets. Files are in use or you have insufficient permissions. Try ending hung Ras.exe processes, closing open programs or rebooting the machine and re-run this cell.") from e

with ThreadPoolExecutor(max_workers=File_Copy_Threads) as executor:
    for remote_full in HECRAS_Deploy_Targets:
        executor.submit(clear_directory, remote_full)


# Copy Folders to HECRAS_Deploy_Targets (with filtering)

# Function to ignore files
def ignore_files(extensions):
    def _ignore_files(pathname, names):
        ignore = set()
        for extension in extensions:
            ignore.update({name for name in names if fnmatch.fnmatch(name, extension)})
        return ignore
    return _ignore_files


# Function to count files in a directory
def count_files_in_directory(directory):
    return sum([len(files) for _, _, files in os.walk(directory)])


# Function for each remote target folder's copy operation
def copy_directory_to_remote(src, dst, num_copies):
    try:
        total_copied_files = 0
        for i in range(1, num_copies + 1):
            new_folder = os.path.join(dst, os.path.basename(src) + '_' + str(i))
            print('\nStarting Copy:               ', new_folder)
            if os.path.exists(new_folder) and response.lower() == 'yes':
                shutil.rmtree(new_folder, ignore_errors=True)
            shutil.copytree(src, new_folder, ignore=ignore_files(exclusions), dirs_exist_ok=True)
            total_copied_files += count_files_in_directory(new_folder)
            time.sleep(1)
        
        return total_copied_files
    except Exception as e:
        print(f"Exception in copy_directory_to_remote for destination {dst}: {e}")
        return 0  # Return 0 so that we avoid the TypeError but still highlight an error


def copy_directory_to_remote_parallel(src, dsts, num_copies):
    if dsts:
        with ThreadPoolExecutor(max_workers=File_Copy_Threads) as executor:
            futures_to_dst = {executor.submit(copy_directory_to_remote, src, dst, num_copies): dst for dst in dsts}
            for future in as_completed(futures_to_dst):
                dst_for_this_future = futures_to_dst[future]
                print("")
                print(f"Finished Deploying:           {dst_for_this_future} with {Number_Parallel_Runs} Parallel Runs {future.result()} files copied.")
        print("\nAll RAS Project Folders are Deployed.")
    else:
        print("No destination directories provided.")



# Call Function to copy from HECRAS_project_folder to HECRAS_Deploy_Targets
copy_directory_to_remote_parallel(HECRAS_project_folder, HECRAS_Deploy_Targets, Number_Parallel_Runs)

In [ ]:
#11 Create Absolute Path Batch File for each Plan File in all Target Directories

# Function to create batch files in the Target Directories
def RAS_Cmdr(hecras_project_directory):
  
    # Scanning the RAS Project Directory for .rasmap file to infer project name
    # Since .prj files are also used by shapefiles, we use a file extension unique to HEC-RAS
    for file_name in os.listdir(hecras_project_directory):
        # Check if the file has .rasmap extension
        if file_name.endswith('.rasmap'):
            # Return the file name without the .rasmap extension
            hecrasproject_prj_filename = file_name[:-7] 
            # Print Message if no .rasmap file is found
            print("")
            print("HEC-RAS Project Name: " + hecrasproject_prj_filename)
            print("") 
 
    # Convert Remote Paths for local execution using Remote_File_Share_Path
    if hecras_project_directory.startswith('\\\\'):
        hecras_project_directory_absolute = Remote_File_Share_Path + hecras_project_directory[hecras_project_directory.find('\\', 2):]
        # print("DEBUG Remote_File_Share_Path: ", Remote_File_Share_Path)
        print(hecras_project_directory[hecras_project_directory.find('\\', 2):])
        print("")
        print("*****  HEC-RAS Path is in a remote file share.  *****")
        print("Converted :" + hecras_project_directory)
        print ("       to: " + hecras_project_directory_absolute)
        
    else:
        hecras_project_directory_absolute = hecras_project_directory
    

    # Target HEC-RAS Project File Path (For script execution)
    RAS_Cmdr_proj_filename = os.path.join(hecras_project_directory, HECRAS_project_name + '.prj')

    # Target HEC-RAS Project File Path (Absolute path for writing the batch file)
    RAS_Cmdr_proj_filename_absolute = os.path.join(hecras_project_directory_absolute, HECRAS_project_name + '.prj')


    # Check if the project file exists
    if os.path.exists(RAS_Cmdr_proj_filename):
        print("")
        print(f"Reading Project File at {RAS_Cmdr_proj_filename}")
    else:
        print("")
        print(f"Project file not found at {RAS_Cmdr_proj_filename}")
        return
    
    # Read the lines of the project file and get the plan file numbers
    with open(RAS_Cmdr_proj_filename, 'r') as f:
        lines = f.readlines()
        plan_lines = [line.strip().split('=')[1] for line in lines if line.startswith('Plan File=')]

    # Create a DataFrame to display the plan files
    df = pd.DataFrame(plan_lines, columns=['Plan Files'])

    print("The PRJ file contains the following plans:")
    print("")
    print(df)
    print("")
    print("Now Batch Files will be created for each plan file.")
    print("")

    # Create batch files for each plan file
    for plan_line in plan_lines:
        
        #Increment the plan number
        plan_number = plan_line.split('=')[-1] 

        print ("------------------------------------")
        print ("Current plan number: " + plan_number)
        
        # Assemble the full HEC-RAS Batch File Command
        # RAS_Cmdr_proj_filename is the same for all plan files, so it is outside the loop
        RAS_Cmdr_plan_filename = os.path.join(hecras_project_directory, HECRAS_project_name + '.' + plan_number)
        RAS_Cmdr_plan_filename_absolute = os.path.join(hecras_project_directory_absolute, HECRAS_project_name + '.' + plan_number)
        # Check if the plan file exists
        if not os.path.exists(RAS_Cmdr_plan_filename):
            # Notify the user if the plan file is not found
            print(f"Plan file {plan_number} not found at {RAS_Cmdr_plan_filename}")
            continue

        # Print the paths for debugging
        print ("HEC-RAS Executable Path: ", hecras_exe_path)
        print ("Current Project File Path: ", RAS_Cmdr_proj_filename)
        print ("Current Plan File Path:    ", RAS_Cmdr_plan_filename)
        
        # Name Batch File per HEC-RAS Project Name and Plan Number
        RAS_Cmdr_bat_file_name = HECRAS_project_name + '.' + plan_number + '.run.bat'
        print ("RAS_Cmdr_bat_file_name: ", RAS_Cmdr_bat_file_name)

        # Assemble the full HEC-RAS Batch File Path (for writing)
        RAS_Cmdr_bat_file_path = os.path.join(hecras_project_directory, RAS_Cmdr_bat_file_name)

        # Assemble the full HEC-RAS Batch File Command
        RAS_Cmdr_bat_file_command = f'"{hecras_exe_path}" -c "{RAS_Cmdr_proj_filename_absolute}" "{RAS_Cmdr_plan_filename_absolute}"'
        # THIS COMMAND CAN BE RUN WITH SUBPROCESS.CALL USING PSEXEC TO RUN ON REMOTE MACHINE
        print(f"BAT File Command: {RAS_Cmdr_bat_file_command}")
     
        # Write the Batch file for this plan file
        with open(RAS_Cmdr_bat_file_path, 'w') as f:
            f.write(RAS_Cmdr_bat_file_command)
        print("")
        print(f"Successfully created batch file at {RAS_Cmdr_bat_file_path}")
        print("")

# Iterate over HECRAS_Deploy_Targets.  For each subdirectory of HECRAS_Deploy_Targets, call RAS_Cmdr
for remote_full in HECRAS_Deploy_Targets:
    for remote_sub in os.listdir(remote_full):
        print ("------------------ RAS Commander: Command Line is All You Need ------------------")
        print ("")
        print ("RAS_Cmdr called for: ", os.path.join(remote_full, remote_sub))
        RAS_Cmdr(os.path.join(remote_full, remote_sub))
        print ("RAS_Cmdr Completed Building BAT FIles for: ", os.path.join(remote_full, remote_sub))
        print ("")
        print ("")
        print ("")

In [ ]:
#12 Queue and Execute each plan via batch file


failed_files = []
running_threads = []

def execute_bat(queue, folder, folder_lock):
    while True:
        try:
            bat_file = queue.get(block=False)
        except Empty:
            break

        # Define the path to the batch file
        bat_path = os.path.join(folder, bat_file)
        print("")

        # Workflow if folder is on remote machine   
        if folder.startswith('\\\\'):
            
            # Replace from "\\" to the next "\" with the Remote_File_Share_Path
            bat_path = Remote_File_Share_Path + folder[folder.find('\\', 2):] + "\\" + bat_file
            

            # For remote paths, grab remote machine 
            # print("Folder: ", folder)
            remote_machine = folder.split('\\', 3)[2]
            print("remote_machine: ", remote_machine)

            # This is the remote command that will be executed with PSEXEC
            if Psexec_Run_In_System_Account == "Yes":
                # For the first condition (running as System account without interaction)
                psexec_cmd = f'{psexec_path} \\\\{remote_machine} -u {username} -p {password} -s -{Psexec_Priority} -accepteula powershell -windowstyle hidden -command "& \'{bat_path}\'"'
            else: 
                # If not "Yes, run interactive with the specified Session ID"
                psexec_cmd = f'{psexec_path} \\\\{remote_machine} -u {username} -p {password} -i {Psexec_Session_ID} -{Psexec_Priority} -accepteula powershell -windowstyle hidden -command "& \'{bat_path}\'"'

            obfuscated_password = "********"
        
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
            # Print the command with the obfuscated password by replacing it only for the print statement
            print(f"[{timestamp}] Executing command: {psexec_cmd.replace(password, obfuscated_password)}")

            with folder_lock:  # Ensure that only one thread can access this subfolder at a time
                try:
                    print(f"[{timestamp}] Running {bat_file} in {folder}")
                    subprocess.check_call(psexec_cmd, shell=True)
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp after command execution
                    print(f"[{timestamp}] Finished {bat_file} in {folder}")
                    
                    # Copy DSS output files after thread execution (Deprecated)
                    # find_and_copy_files(folder, dss_archive_directory, dss_working_directory, dss_file_prefix)
                    
                    print("  ")
                except subprocess.CalledProcessError as e:
                    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp if an error occurs
                    print(f"[{timestamp}] Failed to run {bat_file} in {folder}. Error: {str(e).replace(password, obfuscated_password)}")
                    failed_files.append(bat_file)
        else:
            print("Local Execution")
            try:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                print(f"[{timestamp}] Running {bat_file} locally in {folder}")
                subprocess.call(bat_path)  # Execute the .bat file locally
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp after command execution
                print(f"[{timestamp}] Finished {bat_file} locally in {folder}")
                print("  ")
            except Exception as e:
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Update timestamp if an error occurs
                print(f"[{timestamp}] Failed to run {bat_file} locally in {folder}. \n Error: {str(e)}")
                failed_files.append(bat_file)
            

# Assuming Remote_File_Share_Path is defined somewhere outside the function in the global scope
# Remote_File_Share_Path = "..."

def get_remote_subfolders(target_folders):
    """
    This function returns a list of subfolders for the given remote folders. 
    If a remote subfolder's path starts with '\\', it corrects the path 
    using the globally defined Remote_File_Share_Path.

    Parameters:
    - target_folders (list): A list of paths to the remote folders.

    Returns:
    - list: A list of corrected subfolder paths.
    """
    print(f"target_folders: {target_folders}")
    # Loop through each remote folder in the target folders list
    for remote_folder in target_folders:
        # Only process paths starting with '\\'
        if not remote_folder.startswith("\\\\"):
            print(f"Skipping local path: {remote_folder}")
            continue

        try:
            # Generate a list of all subfolders for the current remote folder
            subfolders = [
                os.path.join(remote_folder, dir)  # Join the remote folder path with the directory name
                for dir in os.listdir(remote_folder)  # List all items in the current remote folder
                if os.path.isdir(os.path.join(remote_folder, dir))  # Check if the item is a directory (subfolder)
            ]
            
            # Append the found subfolders to the remote_subfolders list
            remote_subfolders.extend(subfolders)
            
        except Exception as e:
            # Print any errors encountered when accessing the remote folders
            print(f"Error accessing {remote_folder}: {e}")
            
    # Return the list of corrected subfolder paths
    return remote_subfolders

# Initialize an empty list to store the remote subfolders
remote_subfolders = []

# Get the remote subfolders
remote_subfolders = get_remote_subfolders(HECRAS_Deploy_Targets)


# Now extend the list with local paths from HECRAS_Deploy_Targets
# This but subfolders of HECRAS_Deploy_Targets:  remote_subfolders.extend([path for path in HECRAS_Deploy_Targets if not path.startswith("\\\\")])
for directory in HECRAS_Deploy_Targets:
    if not directory.startswith("\\\\"):
        subfolders = [os.path.join(directory, subfolder) for subfolder in os.listdir(directory) if os.path.isdir(os.path.join(directory, subfolder))]
        remote_subfolders.extend(subfolders)
        
print(f"remote_subfolders: {remote_subfolders}")
print("")


# Create Run_Missing_df based on Operation_Mode
if Operation_Mode == "Build from DSS":
    # display(new_plan_title_df)
    Run_Missing_df = new_plan_title_df.copy()

    # Extract the .bat file names
    Run_Missing_df['BAT File'] = Run_Missing_df['Plan File Name'] + ".run.bat"
    
    # Add Extra Column (Run Missing) and enter "Y" for all rows (since all are missing under a "Build from DSS" scenario)
    Run_Missing_df['Run Missing'] = "Y"
    
    #print("Run_Missing_df")
    #display(Run_Missing_df)  
else:  # Operation_Mode == "Run Missing"
    # Run_Missing_df = Run_Missing_Runlist_df.copy()

    '''Example Run_Missing_Runlist_df
    Plan File Extension
    0	p02
    1	p10
    2	p12
    '''

    # Scanning First Subfolder of First Remote Folder for .bat files
    First_Subfolder_of_First_Remote_Folder = os.listdir(HECRAS_Deploy_Targets[0])[0]
    print("First_Subfolder_of_First_Remote_Folder: ", First_Subfolder_of_First_Remote_Folder)
    
    # Find .bat files and create bat_files_df
    bat_files_path = os.path.join(HECRAS_Deploy_Targets[0], First_Subfolder_of_First_Remote_Folder)
    bat_files = [f for f in os.listdir(bat_files_path) if f.endswith('.bat')]
    print("bat_files: ", bat_files)
    
    # Extract Plan File Extensions from .bat files
    plan_file_extensions = [bat_file.split('.run.bat')[0].split('.', 1)[-1] for bat_file in bat_files]
    
    # Create a DataFrame for .bat files
    bat_files_df = pd.DataFrame({
        'Plan File Extension': plan_file_extensions,
        'BAT File': bat_files
    })

    # Merge the dataframes based on Plan File Extension
    merged_df = Run_Missing_Runlist_df.merge(bat_files_df, on='Plan File Extension', how='left')
    #print("merged_df") # debug only
    #display(merged_df) # debug only

    # Assign merged data back to Run_Missing_Runlist_df
    Run_Missing_Runlist_df['BAT File'] = merged_df['BAT File']

    ''' 
    Plan File Extension	BAT File
    0	p02	WF_WestForkCalcasieu.p02.run.bat
    1	p10	WF_WestForkCalcasieu.p10.run.bat
    2	p12	WF_WestForkCalcasieu.p12.run.bat
    '''
    
    # Extract the .bat file names to Run_Missing_df (to match the dataframe produced in Build from DSS)  Add an extra column (Run Missing marked as "Y" or "N") depending on whether it shows up in Run_Missing_Runlist_df)))
    Run_Missing_Runlist_df['BAT File'] = HECRAS_project_name + "." + Run_Missing_Runlist_df['Plan File Extension'] + ".run.bat"
    Run_Missing_Runlist_df['Run Missing'] = "Y"

    # because Run_Missing_df doesn't get built in this mode, we need to use Run_Missing_Runlist_df to build Run_Missing_df
    Run_Missing_df = Run_Missing_Runlist_df.copy()
        
    
    print("Run_Missing_Runlist_df")
    display(Run_Missing_Runlist_df)


#   ------------------------------------  Queue and Execute each plan via batch file  ------------------------------------


# In the fuction below, use new_plan_title_df DSS Output Exists column to determine if the .bat file should be executed.  
# IF Operation_Mode == "Build from DSS", then all .bat files should be executed
# IF Operation_Mode == "Run Missing", there will be no new_plan_title_df dataframe, so the dataframe information will be built from existing output folder

def distribute_bat_files_among_remote_machines(Run_Missing_df, remote_subfolders):
    if not remote_subfolders:
        print("No remote subfolders provided.")
        return []

    queues = [Queue() for _ in remote_subfolders]
    for i, bat_file in enumerate(Run_Missing_df['BAT File']):
        queues[i % len(remote_subfolders)].put(bat_file)
        print(f"Added {bat_file} to queue {i % len(remote_subfolders)}")
    return queues


# Runs a single thread for each queue
def run_remote_threads(queues, execution_subfolders, folder_locks):
    running_threads = []
    for i, (queue, folder, folder_lock) in enumerate(zip(queues, execution_subfolders, folder_locks)):
        print(f"Starting thread {i} using {folder}")
        thread = threading.Thread(target=execute_bat, args=(queue, folder, folder_lock), daemon=True)
        thread.start()
        running_threads.append(thread)
        time.sleep(5)  # Pause for 5 seconds before starting the next thread
    return running_threads

# Main Execution Script Logic




# ALL THIS IS FOR THE "Build from DSS" OPTION
if Operation_Mode == "Build from DSS":
    # display new_plan_title_df
    print("new_plan_title_df")
    display(new_plan_title_df)
    # THIS IS BLANK


    # Get list of .pxx files from the new plan title dataframe
    new_plan_title_pxx_files = new_plan_title_df["Plan File Name"].tolist()
    print(f"new_plan_title_pxx_files: {new_plan_title_pxx_files}")
    print("")
    # new_plan_title_bat_files = new_plan_title_pxx_files, with an added .run.bat extension
    new_plan_title_bat_files = [file + ".run.bat" for file in new_plan_title_pxx_files]
    print(f"new_plan_title_bat_files: {new_plan_title_bat_files}")
    print("")
    # Build missing_bat_files_df from new_plan_title_bat_files and new_plan_title_pxx_files
    missing_bat_files_df = pd.DataFrame({"PXX File": new_plan_title_pxx_files, "Missing BAT File": new_plan_title_bat_files},)




# Missing

# ADDED
if Operation_Mode in ["Run Missing"]:
# initialize missing_bat_files_df
    missing_bat_files_df = Run_Missing_df.copy()
    



print("missing_bat_files_df")
display(missing_bat_files_df)



# Define queues for each remote machine

# Distribute the missing bat files among the queues
queues = distribute_bat_files_among_remote_machines(Run_Missing_df, remote_subfolders)
print(f"queues: {queues}")

# Create a lock for each subfolder
print(f"queues: {queues}")  
folder_locks = [threading.Lock() for _ in remote_subfolders]

# execution_subfolders is a copy of remote_subfolders and also includes local paths from HECRAS_Deploy_Targets
# Original Line: execution_subfolders = remote_subfolders.copy()
execution_subfolders = remote_subfolders.copy()
print(f"execution_subfolders: {execution_subfolders}")


# Initialize and start multiple threads based on the number of remote machines

# Loop ensures that the main program waits for each thread in running_threads to complete its execution before proceeding
running_threads = run_remote_threads(queues, execution_subfolders, folder_locks)
for thread in running_threads:
    thread.join()

# Notify the user that all .bat files have been executed
print("All .bat files executed.")

# Print the list of files that failed to run
if failed_files:
    print("The following files failed to run:")
    for file in failed_files:
        print(file)

In [ ]:
#13 RESULTS POSTPROCESSING:  Copy all matching files from the list of directories to a single directory, replace if newer  Then, propmt user and delete temporary directories.

def get_remote_subfolders(target_folders):
    """
    This function returns a list of subfolders for the given remote folders. 
    If a remote subfolder's path starts with '\\', it corrects the path 
    using the globally defined Remote_File_Share_Path.

    Parameters:
    - target_folders (list): A list of paths to the remote folders.

    Returns:
    - list: A list of corrected subfolder paths.
    """
    print(f"target_folders: {target_folders}")
    # Loop through each remote folder in the target folders list
    for remote_folder in target_folders:
        # Only process paths starting with '\\'
        if not remote_folder.startswith("\\\\"):
            #print(f"Skipping local path: {remote_folder}")
            continue

        try:
            # Generate a list of all subfolders for the current remote folder
            subfolders = [
                os.path.join(remote_folder, dir)  # Join the remote folder path with the directory name
                for dir in os.listdir(remote_folder)  # List all items in the current remote folder
                if os.path.isdir(os.path.join(remote_folder, dir))  # Check if the item is a directory (subfolder)
            ]
            
            # Append the found subfolders to the remote_subfolders list
            remote_subfolders.extend(subfolders)
            
        except Exception as e:
            # Print any errors encountered when accessing the remote folders
            print(f"Error accessing {remote_folder}: {e}")
            
    # Return the list of corrected subfolder paths
    return remote_subfolders

# Initialize an empty list to store the remote subfolders



# The code below is duplicated from above.  It is included here to keep the code self-contained in case a user runs this cell by itself.

# Initialize an empty list to store the remote subfolders
remote_subfolders = []
# Get the remote subfolders
remote_subfolders = get_remote_subfolders(HECRAS_Deploy_Targets)
# Now extend the list with local paths from HECRAS_Deploy_Targets
# This but subfolders of HECRAS_Deploy_Targets:  remote_subfolders.extend([path for path in HECRAS_Deploy_Targets if not path.startswith("\\\\")])
for directory in HECRAS_Deploy_Targets:
    if not directory.startswith("\\\\"):
        subfolders = [os.path.join(directory, subfolder) for subfolder in os.listdir(directory) if os.path.isdir(os.path.join(directory, subfolder))]
        remote_subfolders.extend(subfolders)
        
print(f"remote_subfolders: {remote_subfolders}")
print("")


# Additional/Optional Settings for Results Collection and INCLUDE/EXCLUDE Filters

# DSS Search Filter Word - useful for filtering out files that are not needed.  Only applies to DSS files.
# NEED TO REVISE FUNCTIONALITY PER ABOVE
DSS_Search_Filter_Include = ""
print("DSS_Search_Filter_Include: " + DSS_Search_Filter_Include)


# Exclude Search Term (e.g. "HMS" to prevent HMS DSS files from being copied at this step)
DSS_Search_Filter_Exclude = ""
print("DSS_Search_Filter_Exclude: " + DSS_Search_Filter_Exclude)


# File Extensions to Include in Search.  Numbered extensions should be placed in second half of list.
file_extensions = ["dss", "hdf", "rst"] + [f"{prefix}{i:02}" for prefix in ["b", "c", "x", "r", "bco"] for i in range(1, 100)]
print("File Extensions: " + str(file_extensions))

### HEC-RAS File Extensions and Descriptions:
### A quick primer on file types that you may be interested in collecting
# 1. **.dss: HEC-RAS DSS output file.
# 2. **.rst: RAS restart file
# 3. **.bXX: Boundary Conditions ASCII input file
# 4. **.cXX: Geometry Preprocessor Output Binary
# 5. **.xXX: Geometry ASCII input file
# 6. **.IC.OXX: Initial Condition binary
# 7. **.gXX.hdf: Geometry in HDF format
# 8. **.pXX.hdf: RAS results in HDF format
# 9. **.tmp.hdf: Temp File with Incomplete RAS results in HDF format
# An example with all file types is shown below:
# file_extensions = ["dss", "hdf", "rst"] + [f"{prefix}{i:02}" for prefix in ["b", "c", "x", "r", "O", "IC.O", "bco"] for i in range(1, 100)]

# NOTE: **.tmp.hdf files are excluded in the code below because they are not complete results files.

# Function to copy a single file, replace if it's newer
def copy_file(source_file, destination_file):
    try:
        if os.path.exists(destination_file):
            if os.path.getmtime(source_file) > os.path.getmtime(destination_file):
                shutil.copy2(source_file, destination_file)
                print(f"Copied {source_file} to {destination_file} (replaced due to being newer).")
                return True
            else:
                dummyvar = 1 #prevents error if debug line is commented out
                #print(f"Skipped {source_file} as the destination file is newer or the same age.")
        else:
            shutil.copy2(source_file, destination_file)
            print(f"Copied {source_file} to {destination_file}.")
            return True
    except PermissionError:
        print(f"Permission denied when attempting to copy {source_file}. Moving to next file.")
    return False

# Check if HECRAS_project_folder is a file
if os.path.isfile(HECRAS_project_folder):
    print(f"Error: {HECRAS_project_folder} is a file. Please delete the file or choose a different output directory.")

files_copied = 0
print("Copying files from the following remote subfolders:")
print(remote_subfolders)

for source_directory in remote_subfolders:
    print(f"Processing source directory: {source_directory}")
    
    if os.path.basename(source_directory).startswith(Folder_Safety_Prefix):
        for root, dirs, files in os.walk(source_directory):
            # Exclude files with "terrain" in the path
            if "terrain" not in root:
                for file in files:
                    # .tmp.hdf files are temporary files created by HEC-RAS, and usually indicate that the model is still running or failed to run
                    # We explicitly exclude these files from the copy process
                    if any(re.search(fr"\.{extension}$", file) and DSS_Search_Filter_Include in file for extension in file_extensions) and not file.endswith('.tmp.hdf'):
                        source_file = os.path.join(root, file)
                        destination_file = os.path.join(HECRAS_project_folder, file)
                        if copy_file(source_file, destination_file):
                            files_copied += 1
                    else:
                        dummyvar = 1 #prevents error if debug line is commented out
                        # print(f"Skipped {file} due to not matching the search criteria.")
            else:
                print(f"Skipped directory {root} due to containing 'terrain'.")
    else:
        print(f"Skipped source directory {source_directory} due to not matching safety prefix.")

if files_copied == 0:
    print("Error: No files found to copy.")
else:
    print(f"Successfully copied {files_copied} files.")



# Cleanup: delete all deployed folders from remote machines using safety prefix

# Delete Existing Files and Directories in the Target Folders

# User Confirmation before Deleting Folders
# Use confirm_deletion() instead of input() to avoid accidental deletion

def request_deletion_confirmation():
    # Function to request user confirmation for deletion

    # Create a popup window
    popup = tk.Tk()
    popup.wm_title("Deletion Confirmation")

    # Set the popup to be always on top
    popup.attributes("-topmost", True)

    message = "This action will delete all files, folders, and subfolders in the HEC-RAS deploy targets. Do you want to proceed?"
    label = tk.Label(popup, text=message, wraplength=300)
    label.pack(pady=10)

    def on_confirm():
        popup.destroy()
        print("Deleting all files, folders, and subfolders in the HEC-RAS deploy targets.")
        with ThreadPoolExecutor(max_workers=File_Copy_Threads) as executor:
            for remote_full in HECRAS_Deploy_Targets:
                executor.submit(shutil.rmtree, remote_full, ignore_errors=True)
        print("Deletion completed.")

    def on_cancel():
        popup.destroy()
        print("Operation cancelled by user. Please check your inputs.")

    # Add buttons for user action
    yes_button = tk.Button(popup, text="Yes", command=on_confirm)
    yes_button.pack(side="left", padx=(20, 10), pady=20)

    no_button = tk.Button(popup, text="No", command=on_cancel)
    no_button.pack(side="right", padx=(10, 20), pady=20)

    # Start the GUI event loop
    popup.mainloop()

# Directly call request_deletion_confirmation() without using input for initial confirmation
request_deletion_confirmation()
